In [1]:
import pandas as pd
import numpy as np

Start_Date = 20250101
End_Date = 20250131

master = pd.read_csv('../Files/SPORTWISE/AS ON 31JAN2025/SPORTWISE MASTER.csv')
sales = pd.read_csv('../Files/SPORTWISE/AS ON 31JAN2025/STYLEWISE SALES 01JAN-31JAN.csv')
opening_inventory = pd.read_csv('../Files/SPORTWISE/AS ON 31JAN2025/Inventory_snapshot_20250101.csv')
current_inventory = pd.read_csv('../Files/SPORTWISE/AS ON 31JAN2025/Inventory_snapshot_20250201.csv')

sales = sales[ (sales['master_brand'] == 'HRX by Hrithik Roshan') & (sales['master_category'] == 'Apparel') & 
                ((sales['business_unit'] == "Men's Sports Apparel") | (sales['business_unit'] == "Women's Sports Apparel")) &
                 ( (sales['gender'] == 'Men') | (sales['gender'] == 'Women') ) & (sales['order_created_date'] >= Start_Date) & (sales['order_created_date'] <= End_Date)]



In [2]:
#functions
def getSport(sid):
    dump = master[master['style_id'] == sid]
    if len(dump) == 0:
        return 'NA'
    else:
        sport = dump.iloc[0]['sport']
        return sport
#########################################################
sales['sport'] = sales['style_id'].apply( getSport )
sales = sales[ sales['sport'] != "NA" ]
sales = sales[sales['po_type'] == "OR"]
#########################################################
def getInventory(sid,dump):
    dump = dump[dump['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        inventory = dump['units'].sum()
        return inventory
#########################################################
def getInventoryValue(sid,dump):
    dump = dump[dump['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        inventory_value = dump['value'].sum()
        return inventory_value
#########################################################
master['opening_inventory'] = master['style_id'].apply( lambda sid : getInventory(sid,opening_inventory) )
master['current_inventory'] = master['style_id'].apply( lambda sid : getInventory(sid,current_inventory) )
master['opening_inventory_value'] = master['style_id'].apply( lambda sid : getInventoryValue(sid,opening_inventory) )
master['current_inventory_value'] = master['style_id'].apply( lambda sid : getInventoryValue(sid,current_inventory) )
#########################################################
def getBAGSOpeningInventory(gender,article,sport):
    if sport == '-' and article == '-':
        dump = master[ (master['gender'] == gender )]
    elif sport == '-':
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) ]
    else:
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) & (master['sport'] == sport)]
    if len(dump) == 0:
        return 0
    else:
        inventory = dump['opening_inventory'].sum()
        return inventory
#########################################################
def getBAGSCurrentInventory(gender,article,sport):
    if sport == '-' and article == '-':
        dump = master[ (master['gender'] == gender )  ]
    elif sport == '-':
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) ]
    else:
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) & (master['sport'] == sport)]
    if len(dump) == 0:
        return 0
    else:
        inventory = dump['current_inventory'].sum()
        return inventory
#########################################################
def getBAGSOpeningInventoryValue(gender,article,sport):
    if sport == '-' and article == '-':
        dump = master[ (master['gender'] == gender )]
    elif sport == '-':
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) ]
    else:
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) & (master['sport'] == sport)]
    if len(dump) == 0:
        return 0
    else:
        inventory_value = dump['opening_inventory_value'].sum()
        return round((inventory_value/100000),2)
#########################################################
def getBAGSCurrentInventoryValue(gender,article,sport):
    if sport == '-' and article == '-':
        dump = master[ (master['gender'] == gender )]
    elif sport == '-':
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) ]
    else:
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) & (master['sport'] == sport)]
    if len(dump) == 0:
        return 0
    else:
        inventory_value = dump['current_inventory_value'].sum()
        return round((inventory_value/100000),2)
#########################################################
def getBAGSStyleCount(gender,article,sport):
    if sport == '-' and article == '-':
        dump = master[ (master['gender'] == gender )]
    elif sport == '-':
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) ]
    else:
        dump = master[ (master['gender'] == gender ) & (master['article_type'] == article) & (master['sport'] == sport)]
    if len(dump) == 0:
        return 0
    else:
        count = len(dump['style_id'].unique())
        return count
#########################################################
def getBAGSSalesAmount(gender,article,sport):
    if article == '-' and sport == '-':
        dump = sales[sales['gender'] == gender]
    elif sport == '-':
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article)  ]
    else:
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article) & (sales['sport'] == sport) ]
    if len(dump) == 0:
        return 0
    else:
        amount  = round(( dump['revenue'].sum() / 100000 ),2)
        return amount
#########################################################
def getBAGSSalesQuantity(gender,article,sport):
    if article == '-' and sport == '-':
        dump = sales[sales['gender'] == gender]
    elif sport == '-':
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article)  ]
    else:
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article) & (sales['sport'] == sport) ]
    if len(dump) == 0:
        return 0
    else:
        qty  = dump['quantity'].sum() 
        return qty
#########################################################
def getBAGSTDPercentage(gender,article,sport):
    if article == '-' and sport == '-':
        dump = sales[sales['gender'] == gender]
    elif sport == '-':
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article)  ]
    else:
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article) & (sales['sport'] == sport) ]
    if len(dump) == 0:
        return 0
    else:
        disc  = round(( ( dump['tot_mrp'].sum() - dump['revenue'].sum() ) / ( dump['tot_mrp'].sum() ) ),4)   
        return disc
#########################################################
def getBAGSASP(gender,article,sport):
    if article == '-' and sport == '-':
        dump = sales[sales['gender'] == gender]
    elif sport == '-':
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article)  ]
    else:
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article) & (sales['sport'] == sport) ]
    if len(dump) == 0:
        return 0
    else:
        asp  = round((  dump['revenue'].sum()  /  dump['quantity'].sum()  ))   
        return asp
#########################################################
def royaltyClean(royalty):
    if pd.isnull(royalty) == True:
        return 0
    else:
        return royalty

sales['royalty'] = sales['royalty'].apply(royaltyClean)
#########################################################
def getBAGSGMPercentage(gender,article,sport):
    if article == '-' and sport == '-':
        dump = sales[sales['gender'] == gender]
    elif sport == '-':
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article)  ]
    else:
        dump = sales[ (sales['gender'] == gender) & (sales['article_type'] == article) & (sales['sport'] == sport) ]
    if len(dump) == 0:
        return 0
    else:
        gm  = round(( ( dump['revenue'].sum() - dump['tax'].sum() - dump['purchase_price'].sum() - dump['royalty'].sum() ) / ( dump['revenue'].sum() ) ),4)   
        return gm
#########################################################

In [3]:
#########################################################
#men Current inventory
MSA_JACKETS_KL_CI = getBAGSCurrentInventory('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_CI = getBAGSCurrentInventory('Men','Jackets','Outdoor')
MSA_JACKETS_KR_CI = getBAGSCurrentInventory('Men','Jackets','Running')
MSA_JACKETS_KT_CI = getBAGSCurrentInventory('Men','Jackets','Training')
MSA_JACKETS_CI    = getBAGSCurrentInventory('Men','Jackets','-')

MSA_SHORTS_KL_CI = getBAGSCurrentInventory('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_CI = getBAGSCurrentInventory('Men','Shorts','Running')
MSA_SHORTS_KT_CI = getBAGSCurrentInventory('Men','Shorts','Training')
MSA_SHORTS_KY_CI = getBAGSCurrentInventory('Men','Shorts','Yoga')
MSA_SHORTS_CI    = getBAGSCurrentInventory('Men','Shorts','-')

MSA_SWEATSHIRTS_KL_CI = getBAGSCurrentInventory('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_CI = getBAGSCurrentInventory('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_CI = getBAGSCurrentInventory('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_CI = getBAGSCurrentInventory('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_CI    = getBAGSCurrentInventory('Men','Sweatshirts','-')

MSA_TRACKPANTS_KL_CI = getBAGSCurrentInventory('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_CI = getBAGSCurrentInventory('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_CI = getBAGSCurrentInventory('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_CI = getBAGSCurrentInventory('Men','Track Pants','Yoga')
MSA_TRACKPANTS_CI    = getBAGSCurrentInventory('Men','Track Pants','-')

MSA_TRACKSUITS_KL_CI = getBAGSCurrentInventory('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_CI = getBAGSCurrentInventory('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_CI = getBAGSCurrentInventory('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_CI = getBAGSCurrentInventory('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_CI    = getBAGSCurrentInventory('Men','Tracksuits','-')

MSA_TSHIRTS_KL_CI = getBAGSCurrentInventory('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_CI = getBAGSCurrentInventory('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_CI = getBAGSCurrentInventory('Men','Tshirts','Running')
MSA_TSHIRTS_KT_CI = getBAGSCurrentInventory('Men','Tshirts','Training')
MSA_TSHIRTS_KY_CI = getBAGSCurrentInventory('Men','Tshirts','Yoga')
MSA_TSHIRTS_CI    = getBAGSCurrentInventory('Men','Tshirts','-')

MSA_CI = getBAGSCurrentInventory('Men','-','-')
#women current inventory
WSA_BRA_KL_CI = getBAGSCurrentInventory('Women','Bra','Lifestyle')
WSA_BRA_KR_CI = getBAGSCurrentInventory('Women','Bra','Running')
WSA_BRA_KT_CI = getBAGSCurrentInventory('Women','Bra','Training')
WSA_BRA_KY_CI = getBAGSCurrentInventory('Women','Bra','Yoga')
WSA_BRA_CI    = getBAGSCurrentInventory('Women','Bra','-')

WSA_JACKETS_KL_CI = getBAGSCurrentInventory('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_CI = getBAGSCurrentInventory('Women','Jackets','Outdoor')
WSA_JACKETS_KR_CI = getBAGSCurrentInventory('Women','Jackets','Running')
WSA_JACKETS_KT_CI = getBAGSCurrentInventory('Women','Jackets','Training')
WSA_JACKETS_CI    = getBAGSCurrentInventory('Women','Jackets','-')

WSA_SHORTS_KL_CI = getBAGSCurrentInventory('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_CI = getBAGSCurrentInventory('Women','Shorts','Running')
WSA_SHORTS_KT_CI = getBAGSCurrentInventory('Women','Shorts','Training')
WSA_SHORTS_KY_CI = getBAGSCurrentInventory('Women','Shorts','Yoga')
WSA_SHORTS_CI    = getBAGSCurrentInventory('Women','Shorts','-')

WSA_SWEATSHIRTS_KL_CI = getBAGSCurrentInventory('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_CI = getBAGSCurrentInventory('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_CI = getBAGSCurrentInventory('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_CI = getBAGSCurrentInventory('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_CI    = getBAGSCurrentInventory('Women','Sweatshirts','-')

WSA_TIGHTS_KL_CI = getBAGSCurrentInventory('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_CI = getBAGSCurrentInventory('Women','Tights','Running')
WSA_TIGHTS_KT_CI = getBAGSCurrentInventory('Women','Tights','Training')
WSA_TIGHTS_KY_CI = getBAGSCurrentInventory('Women','Tights','Yoga')
WSA_TIGHTS_CI    = getBAGSCurrentInventory('Women','Tights','-')


WSA_TRACKPANTS_KL_CI = getBAGSCurrentInventory('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_CI = getBAGSCurrentInventory('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_CI = getBAGSCurrentInventory('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_CI = getBAGSCurrentInventory('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_CI = getBAGSCurrentInventory('Women','Track Pants','Yoga')
WSA_TRACKPANTS_CI    = getBAGSCurrentInventory('Women','Track Pants','-')

WSA_TRACKSUITS_KL_CI = getBAGSCurrentInventory('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_CI = getBAGSCurrentInventory('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_CI = getBAGSCurrentInventory('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_CI = getBAGSCurrentInventory('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_CI    = getBAGSCurrentInventory('Women','Tracksuits','-')

WSA_TSHIRTS_KL_CI = getBAGSCurrentInventory('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_CI = getBAGSCurrentInventory('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_CI = getBAGSCurrentInventory('Women','Tshirts','Running')
WSA_TSHIRTS_KT_CI = getBAGSCurrentInventory('Women','Tshirts','Training')
WSA_TSHIRTS_KY_CI = getBAGSCurrentInventory('Women','Tshirts','Yoga')
WSA_TSHIRTS_CI    = getBAGSCurrentInventory('Women','Tshirts','-')

WSA_CI = getBAGSCurrentInventory('Women','-','-')
#########################################################
#men opening inventory
MSA_JACKETS_KL_OI = getBAGSOpeningInventory('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_OI = getBAGSOpeningInventory('Men','Jackets','Outdoor')
MSA_JACKETS_KR_OI = getBAGSOpeningInventory('Men','Jackets','Running')
MSA_JACKETS_KT_OI = getBAGSOpeningInventory('Men','Jackets','Training')
MSA_JACKETS_OI    = getBAGSOpeningInventory('Men','Jackets','-')
MSA_SHORTS_KL_OI = getBAGSOpeningInventory('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_OI = getBAGSOpeningInventory('Men','Shorts','Running')
MSA_SHORTS_KT_OI = getBAGSOpeningInventory('Men','Shorts','Training')
MSA_SHORTS_KY_OI = getBAGSOpeningInventory('Men','Shorts','Yoga')
MSA_SHORTS_OI    = getBAGSOpeningInventory('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_OI = getBAGSOpeningInventory('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_OI = getBAGSOpeningInventory('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_OI = getBAGSOpeningInventory('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_OI = getBAGSOpeningInventory('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_OI    = getBAGSOpeningInventory('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_OI = getBAGSOpeningInventory('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_OI = getBAGSOpeningInventory('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_OI = getBAGSOpeningInventory('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_OI = getBAGSOpeningInventory('Men','Track Pants','Yoga')
MSA_TRACKPANTS_OI    = getBAGSOpeningInventory('Men','Track Pants','-')
MSA_TRACKSUITS_KL_OI = getBAGSOpeningInventory('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_OI = getBAGSOpeningInventory('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_OI = getBAGSOpeningInventory('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_OI = getBAGSOpeningInventory('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_OI    = getBAGSOpeningInventory('Men','Tracksuits','-')
MSA_TSHIRTS_KL_OI = getBAGSOpeningInventory('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_OI = getBAGSOpeningInventory('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_OI = getBAGSOpeningInventory('Men','Tshirts','Running')
MSA_TSHIRTS_KT_OI = getBAGSOpeningInventory('Men','Tshirts','Training')
MSA_TSHIRTS_KY_OI = getBAGSOpeningInventory('Men','Tshirts','Yoga')
MSA_TSHIRTS_OI    = getBAGSOpeningInventory('Men','Tshirts','-')
MSA_OI = getBAGSOpeningInventory('Men','-','-')
#women opening inventory
WSA_BRA_KL_OI = getBAGSOpeningInventory('Women','Bra','Lifestyle')
WSA_BRA_KR_OI = getBAGSOpeningInventory('Women','Bra','Running')
WSA_BRA_KT_OI = getBAGSOpeningInventory('Women','Bra','Training')
WSA_BRA_KY_OI = getBAGSOpeningInventory('Women','Bra','Yoga')
WSA_BRA_OI    = getBAGSOpeningInventory('Women','Bra','-')
WSA_JACKETS_KL_OI = getBAGSOpeningInventory('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_OI = getBAGSOpeningInventory('Women','Jackets','Outdoor')
WSA_JACKETS_KR_OI = getBAGSOpeningInventory('Women','Jackets','Running')
WSA_JACKETS_KT_OI = getBAGSOpeningInventory('Women','Jackets','Training')
WSA_JACKETS_OI    = getBAGSOpeningInventory('Women','Jackets','-')
WSA_SHORTS_KL_OI = getBAGSOpeningInventory('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_OI = getBAGSOpeningInventory('Women','Shorts','Running')
WSA_SHORTS_KT_OI = getBAGSOpeningInventory('Women','Shorts','Training')
WSA_SHORTS_KY_OI = getBAGSOpeningInventory('Women','Shorts','Yoga')
WSA_SHORTS_OI    = getBAGSOpeningInventory('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_OI = getBAGSOpeningInventory('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_OI = getBAGSOpeningInventory('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_OI = getBAGSOpeningInventory('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_OI = getBAGSOpeningInventory('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_OI    = getBAGSOpeningInventory('Women','Sweatshirts','-')
WSA_TIGHTS_KL_OI = getBAGSOpeningInventory('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_OI = getBAGSOpeningInventory('Women','Tights','Running')
WSA_TIGHTS_KT_OI = getBAGSOpeningInventory('Women','Tights','Training')
WSA_TIGHTS_KY_OI = getBAGSOpeningInventory('Women','Tights','Yoga')
WSA_TIGHTS_OI    = getBAGSOpeningInventory('Women','Tights','-')
WSA_TRACKPANTS_KL_OI = getBAGSOpeningInventory('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_OI = getBAGSOpeningInventory('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_OI = getBAGSOpeningInventory('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_OI = getBAGSOpeningInventory('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_OI = getBAGSOpeningInventory('Women','Track Pants','Yoga')
WSA_TRACKPANTS_OI    = getBAGSOpeningInventory('Women','Track Pants','-')
WSA_TRACKSUITS_KL_OI = getBAGSOpeningInventory('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_OI = getBAGSOpeningInventory('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_OI = getBAGSOpeningInventory('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_OI = getBAGSOpeningInventory('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_OI    = getBAGSOpeningInventory('Women','Tracksuits','-')
WSA_TSHIRTS_KL_OI = getBAGSOpeningInventory('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_OI = getBAGSOpeningInventory('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_OI = getBAGSOpeningInventory('Women','Tshirts','Running')
WSA_TSHIRTS_KT_OI = getBAGSOpeningInventory('Women','Tshirts','Training')
WSA_TSHIRTS_KY_OI = getBAGSOpeningInventory('Women','Tshirts','Yoga')
WSA_TSHIRTS_OI    = getBAGSOpeningInventory('Women','Tshirts','-')
WSA_OI = getBAGSOpeningInventory('Women','-','-')
#########################################################
MSA_JACKETS_KL_CIV = getBAGSCurrentInventoryValue('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_CIV = getBAGSCurrentInventoryValue('Men','Jackets','Outdoor')
MSA_JACKETS_KR_CIV = getBAGSCurrentInventoryValue('Men','Jackets','Running')
MSA_JACKETS_KT_CIV = getBAGSCurrentInventoryValue('Men','Jackets','Training')
MSA_JACKETS_CIV    = getBAGSCurrentInventoryValue('Men','Jackets','-')
MSA_SHORTS_KL_CIV = getBAGSCurrentInventoryValue('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_CIV = getBAGSCurrentInventoryValue('Men','Shorts','Running')
MSA_SHORTS_KT_CIV = getBAGSCurrentInventoryValue('Men','Shorts','Training')
MSA_SHORTS_KY_CIV = getBAGSCurrentInventoryValue('Men','Shorts','Yoga')
MSA_SHORTS_CIV    = getBAGSCurrentInventoryValue('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_CIV = getBAGSCurrentInventoryValue('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_CIV = getBAGSCurrentInventoryValue('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_CIV = getBAGSCurrentInventoryValue('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_CIV = getBAGSCurrentInventoryValue('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_CIV    = getBAGSCurrentInventoryValue('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_CIV = getBAGSCurrentInventoryValue('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_CIV = getBAGSCurrentInventoryValue('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_CIV = getBAGSCurrentInventoryValue('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_CIV = getBAGSCurrentInventoryValue('Men','Track Pants','Yoga')
MSA_TRACKPANTS_CIV    = getBAGSCurrentInventoryValue('Men','Track Pants','-')
MSA_TRACKSUITS_KL_CIV = getBAGSCurrentInventoryValue('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_CIV = getBAGSCurrentInventoryValue('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_CIV = getBAGSCurrentInventoryValue('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_CIV = getBAGSCurrentInventoryValue('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_CIV    = getBAGSCurrentInventoryValue('Men','Tracksuits','-')
MSA_TSHIRTS_KL_CIV = getBAGSCurrentInventoryValue('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_CIV = getBAGSCurrentInventoryValue('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_CIV = getBAGSCurrentInventoryValue('Men','Tshirts','Running')
MSA_TSHIRTS_KT_CIV = getBAGSCurrentInventoryValue('Men','Tshirts','Training')
MSA_TSHIRTS_KY_CIV = getBAGSCurrentInventoryValue('Men','Tshirts','Yoga')
MSA_TSHIRTS_CIV    = getBAGSCurrentInventoryValue('Men','Tshirts','-')
MSA_CIV = getBAGSCurrentInventoryValue('Men','-','-')
WSA_BRA_KL_CIV = getBAGSCurrentInventoryValue('Women','Bra','Lifestyle')
WSA_BRA_KR_CIV = getBAGSCurrentInventoryValue('Women','Bra','Running')
WSA_BRA_KT_CIV = getBAGSCurrentInventoryValue('Women','Bra','Training')
WSA_BRA_KY_CIV = getBAGSCurrentInventoryValue('Women','Bra','Yoga')
WSA_BRA_CIV    = getBAGSCurrentInventoryValue('Women','Bra','-')
WSA_JACKETS_KL_CIV = getBAGSCurrentInventoryValue('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_CIV = getBAGSCurrentInventoryValue('Women','Jackets','Outdoor')
WSA_JACKETS_KR_CIV = getBAGSCurrentInventoryValue('Women','Jackets','Running')
WSA_JACKETS_KT_CIV = getBAGSCurrentInventoryValue('Women','Jackets','Training')
WSA_JACKETS_CIV    = getBAGSCurrentInventoryValue('Women','Jackets','-')
WSA_SHORTS_KL_CIV = getBAGSCurrentInventoryValue('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_CIV = getBAGSCurrentInventoryValue('Women','Shorts','Running')
WSA_SHORTS_KT_CIV = getBAGSCurrentInventoryValue('Women','Shorts','Training')
WSA_SHORTS_KY_CIV = getBAGSCurrentInventoryValue('Women','Shorts','Yoga')
WSA_SHORTS_CIV    = getBAGSCurrentInventoryValue('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_CIV = getBAGSCurrentInventoryValue('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_CIV = getBAGSCurrentInventoryValue('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_CIV = getBAGSCurrentInventoryValue('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_CIV = getBAGSCurrentInventoryValue('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_CIV    = getBAGSCurrentInventoryValue('Women','Sweatshirts','-')
WSA_TIGHTS_KL_CIV = getBAGSCurrentInventoryValue('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_CIV = getBAGSCurrentInventoryValue('Women','Tights','Running')
WSA_TIGHTS_KT_CIV = getBAGSCurrentInventoryValue('Women','Tights','Training')
WSA_TIGHTS_KY_CIV = getBAGSCurrentInventoryValue('Women','Tights','Yoga')
WSA_TIGHTS_CIV    = getBAGSCurrentInventoryValue('Women','Tights','-')
WSA_TRACKPANTS_KL_CIV = getBAGSCurrentInventoryValue('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_CIV = getBAGSCurrentInventoryValue('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_CIV = getBAGSCurrentInventoryValue('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_CIV = getBAGSCurrentInventoryValue('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_CIV = getBAGSCurrentInventoryValue('Women','Track Pants','Yoga')
WSA_TRACKPANTS_CIV    = getBAGSCurrentInventoryValue('Women','Track Pants','-')
WSA_TRACKSUITS_KL_CIV = getBAGSCurrentInventoryValue('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_CIV = getBAGSCurrentInventoryValue('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_CIV = getBAGSCurrentInventoryValue('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_CIV = getBAGSCurrentInventoryValue('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_CIV    = getBAGSCurrentInventoryValue('Women','Tracksuits','-')
WSA_TSHIRTS_KL_CIV = getBAGSCurrentInventoryValue('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_CIV = getBAGSCurrentInventoryValue('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_CIV = getBAGSCurrentInventoryValue('Women','Tshirts','Running')
WSA_TSHIRTS_KT_CIV = getBAGSCurrentInventoryValue('Women','Tshirts','Training')
WSA_TSHIRTS_KY_CIV = getBAGSCurrentInventoryValue('Women','Tshirts','Yoga')
WSA_TSHIRTS_CIV    = getBAGSCurrentInventoryValue('Women','Tshirts','-')
WSA_CIV = getBAGSCurrentInventoryValue('Women','-','-')
#########################################################
MSA_JACKETS_KL_OIV = getBAGSOpeningInventoryValue('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_OIV = getBAGSOpeningInventoryValue('Men','Jackets','Outdoor')
MSA_JACKETS_KR_OIV = getBAGSOpeningInventoryValue('Men','Jackets','Running')
MSA_JACKETS_KT_OIV = getBAGSOpeningInventoryValue('Men','Jackets','Training')
MSA_JACKETS_OIV    = getBAGSOpeningInventoryValue('Men','Jackets','-')
MSA_SHORTS_KL_OIV = getBAGSOpeningInventoryValue('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_OIV = getBAGSOpeningInventoryValue('Men','Shorts','Running')
MSA_SHORTS_KT_OIV = getBAGSOpeningInventoryValue('Men','Shorts','Training')
MSA_SHORTS_KY_OIV = getBAGSOpeningInventoryValue('Men','Shorts','Yoga')
MSA_SHORTS_OIV    = getBAGSOpeningInventoryValue('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_OIV = getBAGSOpeningInventoryValue('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_OIV = getBAGSOpeningInventoryValue('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_OIV = getBAGSOpeningInventoryValue('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_OIV = getBAGSOpeningInventoryValue('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_OIV    = getBAGSOpeningInventoryValue('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_OIV = getBAGSOpeningInventoryValue('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_OIV = getBAGSOpeningInventoryValue('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_OIV = getBAGSOpeningInventoryValue('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_OIV = getBAGSOpeningInventoryValue('Men','Track Pants','Yoga')
MSA_TRACKPANTS_OIV    = getBAGSOpeningInventoryValue('Men','Track Pants','-')
MSA_TRACKSUITS_KL_OIV = getBAGSOpeningInventoryValue('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_OIV = getBAGSOpeningInventoryValue('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_OIV = getBAGSOpeningInventoryValue('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_OIV = getBAGSOpeningInventoryValue('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_OIV    = getBAGSOpeningInventoryValue('Men','Tracksuits','-')
MSA_TSHIRTS_KL_OIV = getBAGSOpeningInventoryValue('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_OIV = getBAGSOpeningInventoryValue('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_OIV = getBAGSOpeningInventoryValue('Men','Tshirts','Running')
MSA_TSHIRTS_KT_OIV = getBAGSOpeningInventoryValue('Men','Tshirts','Training')
MSA_TSHIRTS_KY_OIV = getBAGSOpeningInventoryValue('Men','Tshirts','Yoga')
MSA_TSHIRTS_OIV    = getBAGSOpeningInventoryValue('Men','Tshirts','-')
MSA_OIV = getBAGSOpeningInventoryValue('Men','-','-')
WSA_BRA_KL_OIV = getBAGSOpeningInventoryValue('Women','Bra','Lifestyle')
WSA_BRA_KR_OIV = getBAGSOpeningInventoryValue('Women','Bra','Running')
WSA_BRA_KT_OIV = getBAGSOpeningInventoryValue('Women','Bra','Training')
WSA_BRA_KY_OIV = getBAGSOpeningInventoryValue('Women','Bra','Yoga')
WSA_BRA_OIV    = getBAGSOpeningInventoryValue('Women','Bra','-')
WSA_JACKETS_KL_OIV = getBAGSOpeningInventoryValue('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_OIV = getBAGSOpeningInventoryValue('Women','Jackets','Outdoor')
WSA_JACKETS_KR_OIV = getBAGSOpeningInventoryValue('Women','Jackets','Running')
WSA_JACKETS_KT_OIV = getBAGSOpeningInventoryValue('Women','Jackets','Training')
WSA_JACKETS_OIV    = getBAGSOpeningInventoryValue('Women','Jackets','-')
WSA_SHORTS_KL_OIV = getBAGSOpeningInventoryValue('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_OIV = getBAGSOpeningInventoryValue('Women','Shorts','Running')
WSA_SHORTS_KT_OIV = getBAGSOpeningInventoryValue('Women','Shorts','Training')
WSA_SHORTS_KY_OIV = getBAGSOpeningInventoryValue('Women','Shorts','Yoga')
WSA_SHORTS_OIV    = getBAGSOpeningInventoryValue('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_OIV = getBAGSOpeningInventoryValue('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_OIV = getBAGSOpeningInventoryValue('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_OIV = getBAGSOpeningInventoryValue('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_OIV = getBAGSOpeningInventoryValue('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_OIV    = getBAGSOpeningInventoryValue('Women','Sweatshirts','-')
WSA_TIGHTS_KL_OIV = getBAGSOpeningInventoryValue('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_OIV = getBAGSOpeningInventoryValue('Women','Tights','Running')
WSA_TIGHTS_KT_OIV = getBAGSOpeningInventoryValue('Women','Tights','Training')
WSA_TIGHTS_KY_OIV = getBAGSOpeningInventoryValue('Women','Tights','Yoga')
WSA_TIGHTS_OIV    = getBAGSOpeningInventoryValue('Women','Tights','-')
WSA_TRACKPANTS_KL_OIV = getBAGSOpeningInventoryValue('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_OIV = getBAGSOpeningInventoryValue('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_OIV = getBAGSOpeningInventoryValue('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_OIV = getBAGSOpeningInventoryValue('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_OIV = getBAGSOpeningInventoryValue('Women','Track Pants','Yoga')
WSA_TRACKPANTS_OIV    = getBAGSOpeningInventoryValue('Women','Track Pants','-')
WSA_TRACKSUITS_KL_OIV = getBAGSOpeningInventoryValue('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_OIV = getBAGSOpeningInventoryValue('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_OIV = getBAGSOpeningInventoryValue('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_OIV = getBAGSOpeningInventoryValue('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_OIV    = getBAGSOpeningInventoryValue('Women','Tracksuits','-')
WSA_TSHIRTS_KL_OIV = getBAGSOpeningInventoryValue('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_OIV = getBAGSOpeningInventoryValue('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_OIV = getBAGSOpeningInventoryValue('Women','Tshirts','Running')
WSA_TSHIRTS_KT_OIV = getBAGSOpeningInventoryValue('Women','Tshirts','Training')
WSA_TSHIRTS_KY_OIV = getBAGSOpeningInventoryValue('Women','Tshirts','Yoga')
WSA_TSHIRTS_OIV    = getBAGSOpeningInventoryValue('Women','Tshirts','-')
WSA_OIV = getBAGSOpeningInventoryValue('Women','-','-')
#########################################################
MSA_JACKETS_KL_STYLES_COUNT = getBAGSStyleCount('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_STYLES_COUNT = getBAGSStyleCount('Men','Jackets','Outdoor')
MSA_JACKETS_KR_STYLES_COUNT = getBAGSStyleCount('Men','Jackets','Running')
MSA_JACKETS_KT_STYLES_COUNT = getBAGSStyleCount('Men','Jackets','Training')
MSA_JACKETS_STYLES_COUNT    = getBAGSStyleCount('Men','Jackets','-')
MSA_SHORTS_KL_STYLES_COUNT = getBAGSStyleCount('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_STYLES_COUNT = getBAGSStyleCount('Men','Shorts','Running')
MSA_SHORTS_KT_STYLES_COUNT = getBAGSStyleCount('Men','Shorts','Training')
MSA_SHORTS_KY_STYLES_COUNT = getBAGSStyleCount('Men','Shorts','Yoga')
MSA_SHORTS_STYLES_COUNT    = getBAGSStyleCount('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_STYLES_COUNT = getBAGSStyleCount('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_STYLES_COUNT = getBAGSStyleCount('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_STYLES_COUNT = getBAGSStyleCount('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_STYLES_COUNT = getBAGSStyleCount('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_STYLES_COUNT    = getBAGSStyleCount('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_STYLES_COUNT = getBAGSStyleCount('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_STYLES_COUNT = getBAGSStyleCount('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_STYLES_COUNT = getBAGSStyleCount('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_STYLES_COUNT = getBAGSStyleCount('Men','Track Pants','Yoga')
MSA_TRACKPANTS_STYLES_COUNT    = getBAGSStyleCount('Men','Track Pants','-')
MSA_TRACKSUITS_KL_STYLES_COUNT = getBAGSStyleCount('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_STYLES_COUNT = getBAGSStyleCount('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_STYLES_COUNT = getBAGSStyleCount('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_STYLES_COUNT = getBAGSStyleCount('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_STYLES_COUNT    = getBAGSStyleCount('Men','Tracksuits','-')
MSA_TSHIRTS_KL_STYLES_COUNT = getBAGSStyleCount('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_STYLES_COUNT = getBAGSStyleCount('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_STYLES_COUNT = getBAGSStyleCount('Men','Tshirts','Running')
MSA_TSHIRTS_KT_STYLES_COUNT = getBAGSStyleCount('Men','Tshirts','Training')
MSA_TSHIRTS_KY_STYLES_COUNT = getBAGSStyleCount('Men','Tshirts','Yoga')
MSA_TSHIRTS_STYLES_COUNT    = getBAGSStyleCount('Men','Tshirts','-')
MSA_STYLES_COUNT = getBAGSStyleCount('Men','-','-')
WSA_BRA_KL_STYLES_COUNT = getBAGSStyleCount('Women','Bra','Lifestyle')
WSA_BRA_KR_STYLES_COUNT = getBAGSStyleCount('Women','Bra','Running')
WSA_BRA_KT_STYLES_COUNT = getBAGSStyleCount('Women','Bra','Training')
WSA_BRA_KY_STYLES_COUNT = getBAGSStyleCount('Women','Bra','Yoga')
WSA_BRA_STYLES_COUNT    = getBAGSStyleCount('Women','Bra','-')
WSA_JACKETS_KL_STYLES_COUNT = getBAGSStyleCount('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_STYLES_COUNT = getBAGSStyleCount('Women','Jackets','Outdoor')
WSA_JACKETS_KR_STYLES_COUNT = getBAGSStyleCount('Women','Jackets','Running')
WSA_JACKETS_KT_STYLES_COUNT = getBAGSStyleCount('Women','Jackets','Training')
WSA_JACKETS_STYLES_COUNT    = getBAGSStyleCount('Women','Jackets','-')
WSA_SHORTS_KL_STYLES_COUNT = getBAGSStyleCount('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_STYLES_COUNT = getBAGSStyleCount('Women','Shorts','Running')
WSA_SHORTS_KT_STYLES_COUNT = getBAGSStyleCount('Women','Shorts','Training')
WSA_SHORTS_KY_STYLES_COUNT = getBAGSStyleCount('Women','Shorts','Yoga')
WSA_SHORTS_STYLES_COUNT    = getBAGSStyleCount('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_STYLES_COUNT = getBAGSStyleCount('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_STYLES_COUNT = getBAGSStyleCount('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_STYLES_COUNT = getBAGSStyleCount('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_STYLES_COUNT = getBAGSStyleCount('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_STYLES_COUNT    = getBAGSStyleCount('Women','Sweatshirts','-')
WSA_TIGHTS_KL_STYLES_COUNT = getBAGSStyleCount('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_STYLES_COUNT = getBAGSStyleCount('Women','Tights','Running')
WSA_TIGHTS_KT_STYLES_COUNT = getBAGSStyleCount('Women','Tights','Training')
WSA_TIGHTS_KY_STYLES_COUNT = getBAGSStyleCount('Women','Tights','Yoga')
WSA_TIGHTS_STYLES_COUNT    = getBAGSStyleCount('Women','Tights','-')
WSA_TRACKPANTS_KL_STYLES_COUNT = getBAGSStyleCount('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_STYLES_COUNT = getBAGSStyleCount('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_STYLES_COUNT = getBAGSStyleCount('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_STYLES_COUNT = getBAGSStyleCount('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_STYLES_COUNT = getBAGSStyleCount('Women','Track Pants','Yoga')
WSA_TRACKPANTS_STYLES_COUNT    = getBAGSStyleCount('Women','Track Pants','-')
WSA_TRACKSUITS_KL_STYLES_COUNT = getBAGSStyleCount('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_STYLES_COUNT = getBAGSStyleCount('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_STYLES_COUNT = getBAGSStyleCount('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_STYLES_COUNT = getBAGSStyleCount('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_STYLES_COUNT    = getBAGSStyleCount('Women','Tracksuits','-')
WSA_TSHIRTS_KL_STYLES_COUNT = getBAGSStyleCount('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_STYLES_COUNT = getBAGSStyleCount('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_STYLES_COUNT = getBAGSStyleCount('Women','Tshirts','Running')
WSA_TSHIRTS_KT_STYLES_COUNT = getBAGSStyleCount('Women','Tshirts','Training')
WSA_TSHIRTS_KY_STYLES_COUNT = getBAGSStyleCount('Women','Tshirts','Yoga')
WSA_TSHIRTS_STYLES_COUNT    = getBAGSStyleCount('Women','Tshirts','-')
WSA_STYLES_COUNT = getBAGSStyleCount('Women','-','-')
#########################################################
MSA_JACKETS_KL_SALES_AMOUNT = getBAGSSalesAmount('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_SALES_AMOUNT = getBAGSSalesAmount('Men','Jackets','Outdoor')
MSA_JACKETS_KR_SALES_AMOUNT = getBAGSSalesAmount('Men','Jackets','Running')
MSA_JACKETS_KT_SALES_AMOUNT = getBAGSSalesAmount('Men','Jackets','Training')
MSA_JACKETS_SALES_AMOUNT    = getBAGSSalesAmount('Men','Jackets','-')
MSA_SHORTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Men','Shorts','Running')
MSA_SHORTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Men','Shorts','Training')
MSA_SHORTS_KY_SALES_AMOUNT = getBAGSSalesAmount('Men','Shorts','Yoga')
MSA_SHORTS_SALES_AMOUNT    = getBAGSSalesAmount('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_SALES_AMOUNT = getBAGSSalesAmount('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_SALES_AMOUNT    = getBAGSSalesAmount('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_SALES_AMOUNT = getBAGSSalesAmount('Men','Track Pants','Yoga')
MSA_TRACKPANTS_SALES_AMOUNT    = getBAGSSalesAmount('Men','Track Pants','-')
MSA_TRACKSUITS_KL_SALES_AMOUNT = getBAGSSalesAmount('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_SALES_AMOUNT = getBAGSSalesAmount('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_SALES_AMOUNT = getBAGSSalesAmount('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_SALES_AMOUNT = getBAGSSalesAmount('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_SALES_AMOUNT    = getBAGSSalesAmount('Men','Tracksuits','-')
MSA_TSHIRTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_SALES_AMOUNT = getBAGSSalesAmount('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Men','Tshirts','Running')
MSA_TSHIRTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Men','Tshirts','Training')
MSA_TSHIRTS_KY_SALES_AMOUNT = getBAGSSalesAmount('Men','Tshirts','Yoga')
MSA_TSHIRTS_SALES_AMOUNT    = getBAGSSalesAmount('Men','Tshirts','-')
MSA_SALES_AMOUNT = getBAGSSalesAmount('Men','-','-')
WSA_BRA_KL_SALES_AMOUNT = getBAGSSalesAmount('Women','Bra','Lifestyle')
WSA_BRA_KR_SALES_AMOUNT = getBAGSSalesAmount('Women','Bra','Running')
WSA_BRA_KT_SALES_AMOUNT = getBAGSSalesAmount('Women','Bra','Training')
WSA_BRA_KY_SALES_AMOUNT = getBAGSSalesAmount('Women','Bra','Yoga')
WSA_BRA_SALES_AMOUNT    = getBAGSSalesAmount('Women','Bra','-')
WSA_JACKETS_KL_SALES_AMOUNT = getBAGSSalesAmount('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_SALES_AMOUNT = getBAGSSalesAmount('Women','Jackets','Outdoor')
WSA_JACKETS_KR_SALES_AMOUNT = getBAGSSalesAmount('Women','Jackets','Running')
WSA_JACKETS_KT_SALES_AMOUNT = getBAGSSalesAmount('Women','Jackets','Training')
WSA_JACKETS_SALES_AMOUNT    = getBAGSSalesAmount('Women','Jackets','-')
WSA_SHORTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Women','Shorts','Running')
WSA_SHORTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Women','Shorts','Training')
WSA_SHORTS_KY_SALES_AMOUNT = getBAGSSalesAmount('Women','Shorts','Yoga')
WSA_SHORTS_SALES_AMOUNT    = getBAGSSalesAmount('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_SALES_AMOUNT = getBAGSSalesAmount('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_SALES_AMOUNT    = getBAGSSalesAmount('Women','Sweatshirts','-')
WSA_TIGHTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Women','Tights','Running')
WSA_TIGHTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Women','Tights','Training')
WSA_TIGHTS_KY_SALES_AMOUNT = getBAGSSalesAmount('Women','Tights','Yoga')
WSA_TIGHTS_SALES_AMOUNT    = getBAGSSalesAmount('Women','Tights','-')
WSA_TRACKPANTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_SALES_AMOUNT = getBAGSSalesAmount('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_SALES_AMOUNT = getBAGSSalesAmount('Women','Track Pants','Yoga')
WSA_TRACKPANTS_SALES_AMOUNT    = getBAGSSalesAmount('Women','Track Pants','-')
WSA_TRACKSUITS_KL_SALES_AMOUNT = getBAGSSalesAmount('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_SALES_AMOUNT = getBAGSSalesAmount('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_SALES_AMOUNT = getBAGSSalesAmount('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_SALES_AMOUNT = getBAGSSalesAmount('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_SALES_AMOUNT    = getBAGSSalesAmount('Women','Tracksuits','-')
WSA_TSHIRTS_KL_SALES_AMOUNT = getBAGSSalesAmount('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_SALES_AMOUNT = getBAGSSalesAmount('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_SALES_AMOUNT = getBAGSSalesAmount('Women','Tshirts','Running')
WSA_TSHIRTS_KT_SALES_AMOUNT = getBAGSSalesAmount('Women','Tshirts','Training')
WSA_TSHIRTS_KY_SALES_AMOUNT = getBAGSSalesAmount('Women','Tshirts','Yoga')
WSA_TSHIRTS_SALES_AMOUNT    = getBAGSSalesAmount('Women','Tshirts','-')
WSA_SALES_AMOUNT = getBAGSSalesAmount('Women','-','-')
#########################################################
MSA_JACKETS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_SALES_QUANTITY = getBAGSSalesQuantity('Men','Jackets','Outdoor')
MSA_JACKETS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Men','Jackets','Running')
MSA_JACKETS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Men','Jackets','Training')
MSA_JACKETS_SALES_QUANTITY    = getBAGSSalesQuantity('Men','Jackets','-')
MSA_SHORTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Men','Shorts','Running')
MSA_SHORTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Men','Shorts','Training')
MSA_SHORTS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Men','Shorts','Yoga')
MSA_SHORTS_SALES_QUANTITY    = getBAGSSalesQuantity('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_SALES_QUANTITY = getBAGSSalesQuantity('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_SALES_QUANTITY    = getBAGSSalesQuantity('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Men','Track Pants','Yoga')
MSA_TRACKPANTS_SALES_QUANTITY    = getBAGSSalesQuantity('Men','Track Pants','-')
MSA_TRACKSUITS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_SALES_QUANTITY    = getBAGSSalesQuantity('Men','Tracksuits','-')
MSA_TSHIRTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tshirts','Running')
MSA_TSHIRTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tshirts','Training')
MSA_TSHIRTS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Men','Tshirts','Yoga')
MSA_TSHIRTS_SALES_QUANTITY    = getBAGSSalesQuantity('Men','Tshirts','-')
MSA_SALES_QUANTITY = getBAGSSalesQuantity('Men','-','-')
WSA_BRA_KL_SALES_QUANTITY = getBAGSSalesQuantity('Women','Bra','Lifestyle')
WSA_BRA_KR_SALES_QUANTITY = getBAGSSalesQuantity('Women','Bra','Running')
WSA_BRA_KT_SALES_QUANTITY = getBAGSSalesQuantity('Women','Bra','Training')
WSA_BRA_KY_SALES_QUANTITY = getBAGSSalesQuantity('Women','Bra','Yoga')
WSA_BRA_SALES_QUANTITY    = getBAGSSalesQuantity('Women','Bra','-')
WSA_JACKETS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_SALES_QUANTITY = getBAGSSalesQuantity('Women','Jackets','Outdoor')
WSA_JACKETS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Women','Jackets','Running')
WSA_JACKETS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Women','Jackets','Training')
WSA_JACKETS_SALES_QUANTITY    = getBAGSSalesQuantity('Women','Jackets','-')
WSA_SHORTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Women','Shorts','Running')
WSA_SHORTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Women','Shorts','Training')
WSA_SHORTS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Women','Shorts','Yoga')
WSA_SHORTS_SALES_QUANTITY    = getBAGSSalesQuantity('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_SALES_QUANTITY = getBAGSSalesQuantity('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_SALES_QUANTITY    = getBAGSSalesQuantity('Women','Sweatshirts','-')
WSA_TIGHTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tights','Running')
WSA_TIGHTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tights','Training')
WSA_TIGHTS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tights','Yoga')
WSA_TIGHTS_SALES_QUANTITY    = getBAGSSalesQuantity('Women','Tights','-')
WSA_TRACKPANTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_SALES_QUANTITY = getBAGSSalesQuantity('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Women','Track Pants','Yoga')
WSA_TRACKPANTS_SALES_QUANTITY    = getBAGSSalesQuantity('Women','Track Pants','-')
WSA_TRACKSUITS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_SALES_QUANTITY    = getBAGSSalesQuantity('Women','Tracksuits','-')
WSA_TSHIRTS_KL_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tshirts','Running')
WSA_TSHIRTS_KT_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tshirts','Training')
WSA_TSHIRTS_KY_SALES_QUANTITY = getBAGSSalesQuantity('Women','Tshirts','Yoga')
WSA_TSHIRTS_SALES_QUANTITY    = getBAGSSalesQuantity('Women','Tshirts','-')
WSA_SALES_QUANTITY = getBAGSSalesQuantity('Women','-','-')
#########################################################
MSA_JACKETS_KL_TD = getBAGSTDPercentage('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_TD = getBAGSTDPercentage('Men','Jackets','Outdoor')
MSA_JACKETS_KR_TD = getBAGSTDPercentage('Men','Jackets','Running')
MSA_JACKETS_KT_TD = getBAGSTDPercentage('Men','Jackets','Training')
MSA_JACKETS_TD    = getBAGSTDPercentage('Men','Jackets','-')
MSA_SHORTS_KL_TD = getBAGSTDPercentage('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_TD = getBAGSTDPercentage('Men','Shorts','Running')
MSA_SHORTS_KT_TD = getBAGSTDPercentage('Men','Shorts','Training')
MSA_SHORTS_KY_TD = getBAGSTDPercentage('Men','Shorts','Yoga')
MSA_SHORTS_TD    = getBAGSTDPercentage('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_TD = getBAGSTDPercentage('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_TD = getBAGSTDPercentage('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_TD = getBAGSTDPercentage('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_TD = getBAGSTDPercentage('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_TD    = getBAGSTDPercentage('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_TD = getBAGSTDPercentage('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_TD = getBAGSTDPercentage('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_TD = getBAGSTDPercentage('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_TD = getBAGSTDPercentage('Men','Track Pants','Yoga')
MSA_TRACKPANTS_TD    = getBAGSTDPercentage('Men','Track Pants','-')
MSA_TRACKSUITS_KL_TD = getBAGSTDPercentage('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_TD = getBAGSTDPercentage('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_TD = getBAGSTDPercentage('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_TD = getBAGSTDPercentage('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_TD    = getBAGSTDPercentage('Men','Tracksuits','-')
MSA_TSHIRTS_KL_TD = getBAGSTDPercentage('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_TD = getBAGSTDPercentage('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_TD = getBAGSTDPercentage('Men','Tshirts','Running')
MSA_TSHIRTS_KT_TD = getBAGSTDPercentage('Men','Tshirts','Training')
MSA_TSHIRTS_KY_TD = getBAGSTDPercentage('Men','Tshirts','Yoga')
MSA_TSHIRTS_TD    = getBAGSTDPercentage('Men','Tshirts','-')
MSA_TD = getBAGSTDPercentage('Men','-','-')
WSA_BRA_KL_TD = getBAGSTDPercentage('Women','Bra','Lifestyle')
WSA_BRA_KR_TD = getBAGSTDPercentage('Women','Bra','Running')
WSA_BRA_KT_TD = getBAGSTDPercentage('Women','Bra','Training')
WSA_BRA_KY_TD = getBAGSTDPercentage('Women','Bra','Yoga')
WSA_BRA_TD    = getBAGSTDPercentage('Women','Bra','-')
WSA_JACKETS_KL_TD = getBAGSTDPercentage('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_TD = getBAGSTDPercentage('Women','Jackets','Outdoor')
WSA_JACKETS_KR_TD = getBAGSTDPercentage('Women','Jackets','Running')
WSA_JACKETS_KT_TD = getBAGSTDPercentage('Women','Jackets','Training')
WSA_JACKETS_TD    = getBAGSTDPercentage('Women','Jackets','-')
WSA_SHORTS_KL_TD = getBAGSTDPercentage('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_TD = getBAGSTDPercentage('Women','Shorts','Running')
WSA_SHORTS_KT_TD = getBAGSTDPercentage('Women','Shorts','Training')
WSA_SHORTS_KY_TD = getBAGSTDPercentage('Women','Shorts','Yoga')
WSA_SHORTS_TD    = getBAGSTDPercentage('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_TD = getBAGSTDPercentage('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_TD = getBAGSTDPercentage('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_TD = getBAGSTDPercentage('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_TD = getBAGSTDPercentage('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_TD    = getBAGSTDPercentage('Women','Sweatshirts','-')
WSA_TIGHTS_KL_TD = getBAGSTDPercentage('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_TD = getBAGSTDPercentage('Women','Tights','Running')
WSA_TIGHTS_KT_TD = getBAGSTDPercentage('Women','Tights','Training')
WSA_TIGHTS_KY_TD = getBAGSTDPercentage('Women','Tights','Yoga')
WSA_TIGHTS_TD    = getBAGSTDPercentage('Women','Tights','-')
WSA_TRACKPANTS_KL_TD = getBAGSTDPercentage('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_TD = getBAGSTDPercentage('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_TD = getBAGSTDPercentage('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_TD = getBAGSTDPercentage('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_TD = getBAGSTDPercentage('Women','Track Pants','Yoga')
WSA_TRACKPANTS_TD    = getBAGSTDPercentage('Women','Track Pants','-')
WSA_TRACKSUITS_KL_TD = getBAGSTDPercentage('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_TD = getBAGSTDPercentage('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_TD = getBAGSTDPercentage('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_TD = getBAGSTDPercentage('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_TD    = getBAGSTDPercentage('Women','Tracksuits','-')
WSA_TSHIRTS_KL_TD = getBAGSTDPercentage('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_TD = getBAGSTDPercentage('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_TD = getBAGSTDPercentage('Women','Tshirts','Running')
WSA_TSHIRTS_KT_TD = getBAGSTDPercentage('Women','Tshirts','Training')
WSA_TSHIRTS_KY_TD = getBAGSTDPercentage('Women','Tshirts','Yoga')
WSA_TSHIRTS_TD    = getBAGSTDPercentage('Women','Tshirts','-')
WSA_TD = getBAGSTDPercentage('Women','-','-')
#########################################################
MSA_JACKETS_KL_ASP = getBAGSASP('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_ASP = getBAGSASP('Men','Jackets','Outdoor')
MSA_JACKETS_KR_ASP = getBAGSASP('Men','Jackets','Running')
MSA_JACKETS_KT_ASP = getBAGSASP('Men','Jackets','Training')
MSA_JACKETS_ASP    = getBAGSASP('Men','Jackets','-')
MSA_SHORTS_KL_ASP = getBAGSASP('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_ASP = getBAGSASP('Men','Shorts','Running')
MSA_SHORTS_KT_ASP = getBAGSASP('Men','Shorts','Training')
MSA_SHORTS_KY_ASP = getBAGSASP('Men','Shorts','Yoga')
MSA_SHORTS_ASP    = getBAGSASP('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_ASP = getBAGSASP('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_ASP = getBAGSASP('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_ASP = getBAGSASP('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_ASP = getBAGSASP('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_ASP    = getBAGSASP('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_ASP = getBAGSASP('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_ASP = getBAGSASP('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_ASP = getBAGSASP('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_ASP = getBAGSASP('Men','Track Pants','Yoga')
MSA_TRACKPANTS_ASP    = getBAGSASP('Men','Track Pants','-')
MSA_TRACKSUITS_KL_ASP = getBAGSASP('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_ASP = getBAGSASP('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_ASP = getBAGSASP('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_ASP = getBAGSASP('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_ASP    = getBAGSASP('Men','Tracksuits','-')
MSA_TSHIRTS_KL_ASP = getBAGSASP('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_ASP = getBAGSASP('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_ASP = getBAGSASP('Men','Tshirts','Running')
MSA_TSHIRTS_KT_ASP = getBAGSASP('Men','Tshirts','Training')
MSA_TSHIRTS_KY_ASP = getBAGSASP('Men','Tshirts','Yoga')
MSA_TSHIRTS_ASP    = getBAGSASP('Men','Tshirts','-')
MSA_ASP = getBAGSASP('Men','-','-')
WSA_BRA_KL_ASP = getBAGSASP('Women','Bra','Lifestyle')
WSA_BRA_KR_ASP = getBAGSASP('Women','Bra','Running')
WSA_BRA_KT_ASP = getBAGSASP('Women','Bra','Training')
WSA_BRA_KY_ASP = getBAGSASP('Women','Bra','Yoga')
WSA_BRA_ASP    = getBAGSASP('Women','Bra','-')
WSA_JACKETS_KL_ASP = getBAGSASP('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_ASP = getBAGSASP('Women','Jackets','Outdoor')
WSA_JACKETS_KR_ASP = getBAGSASP('Women','Jackets','Running')
WSA_JACKETS_KT_ASP = getBAGSASP('Women','Jackets','Training')
WSA_JACKETS_ASP    = getBAGSASP('Women','Jackets','-')
WSA_SHORTS_KL_ASP = getBAGSASP('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_ASP = getBAGSASP('Women','Shorts','Running')
WSA_SHORTS_KT_ASP = getBAGSASP('Women','Shorts','Training')
WSA_SHORTS_KY_ASP = getBAGSASP('Women','Shorts','Yoga')
WSA_SHORTS_ASP    = getBAGSASP('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_ASP = getBAGSASP('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_ASP = getBAGSASP('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_ASP = getBAGSASP('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_ASP = getBAGSASP('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_ASP    = getBAGSASP('Women','Sweatshirts','-')
WSA_TIGHTS_KL_ASP = getBAGSASP('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_ASP = getBAGSASP('Women','Tights','Running')
WSA_TIGHTS_KT_ASP = getBAGSASP('Women','Tights','Training')
WSA_TIGHTS_KY_ASP = getBAGSASP('Women','Tights','Yoga')
WSA_TIGHTS_ASP    = getBAGSASP('Women','Tights','-')
WSA_TRACKPANTS_KL_ASP = getBAGSASP('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_ASP = getBAGSASP('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_ASP = getBAGSASP('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_ASP = getBAGSASP('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_ASP = getBAGSASP('Women','Track Pants','Yoga')
WSA_TRACKPANTS_ASP    = getBAGSASP('Women','Track Pants','-')
WSA_TRACKSUITS_KL_ASP = getBAGSASP('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_ASP = getBAGSASP('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_ASP = getBAGSASP('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_ASP = getBAGSASP('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_ASP    = getBAGSASP('Women','Tracksuits','-')
WSA_TSHIRTS_KL_ASP = getBAGSASP('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_ASP = getBAGSASP('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_ASP = getBAGSASP('Women','Tshirts','Running')
WSA_TSHIRTS_KT_ASP = getBAGSASP('Women','Tshirts','Training')
WSA_TSHIRTS_KY_ASP = getBAGSASP('Women','Tshirts','Yoga')
WSA_TSHIRTS_ASP    = getBAGSASP('Women','Tshirts','-')
WSA_ASP = getBAGSASP('Women','-','-')
#########################################################
MSA_JACKETS_KL_GM = getBAGSGMPercentage('Men','Jackets','Lifestyle')
MSA_JACKETS_KO_GM = getBAGSGMPercentage('Men','Jackets','Outdoor')
MSA_JACKETS_KR_GM = getBAGSGMPercentage('Men','Jackets','Running')
MSA_JACKETS_KT_GM = getBAGSGMPercentage('Men','Jackets','Training')
MSA_JACKETS_GM    = getBAGSGMPercentage('Men','Jackets','-')
MSA_SHORTS_KL_GM = getBAGSGMPercentage('Men','Shorts','Lifestyle')
MSA_SHORTS_KR_GM = getBAGSGMPercentage('Men','Shorts','Running')
MSA_SHORTS_KT_GM = getBAGSGMPercentage('Men','Shorts','Training')
MSA_SHORTS_KY_GM = getBAGSGMPercentage('Men','Shorts','Yoga')
MSA_SHORTS_GM    = getBAGSGMPercentage('Men','Shorts','-')
MSA_SWEATSHIRTS_KL_GM = getBAGSGMPercentage('Men','Sweatshirts','Lifestyle')
MSA_SWEATSHIRTS_KO_GM = getBAGSGMPercentage('Men','Sweatshirts','Outdoor')
MSA_SWEATSHIRTS_KR_GM = getBAGSGMPercentage('Men','Sweatshirts','Running')
MSA_SWEATSHIRTS_KT_GM = getBAGSGMPercentage('Men','Sweatshirts','Training')
MSA_SWEATSHIRTS_GM    = getBAGSGMPercentage('Men','Sweatshirts','-')
MSA_TRACKPANTS_KL_GM = getBAGSGMPercentage('Men','Track Pants','Lifestyle')
MSA_TRACKPANTS_KR_GM = getBAGSGMPercentage('Men','Track Pants','Running')
MSA_TRACKPANTS_KT_GM = getBAGSGMPercentage('Men','Track Pants','Training')
MSA_TRACKPANTS_KY_GM = getBAGSGMPercentage('Men','Track Pants','Yoga')
MSA_TRACKPANTS_GM    = getBAGSGMPercentage('Men','Track Pants','-')
MSA_TRACKSUITS_KL_GM = getBAGSGMPercentage('Men','Tracksuits','Lifestyle')
MSA_TRACKSUITS_KR_GM = getBAGSGMPercentage('Men','Tracksuits','Running')
MSA_TRACKSUITS_KT_GM = getBAGSGMPercentage('Men','Tracksuits','Training')
MSA_TRACKSUITS_KY_GM = getBAGSGMPercentage('Men','Tracksuits','Yoga')
MSA_TRACKSUITS_GM    = getBAGSGMPercentage('Men','Tracksuits','-')
MSA_TSHIRTS_KL_GM = getBAGSGMPercentage('Men','Tshirts','Lifestyle')
MSA_TSHIRTS_KO_GM = getBAGSGMPercentage('Men','Tshirts','Outdoor')
MSA_TSHIRTS_KR_GM = getBAGSGMPercentage('Men','Tshirts','Running')
MSA_TSHIRTS_KT_GM = getBAGSGMPercentage('Men','Tshirts','Training')
MSA_TSHIRTS_KY_GM = getBAGSGMPercentage('Men','Tshirts','Yoga')
MSA_TSHIRTS_GM    = getBAGSGMPercentage('Men','Tshirts','-')
MSA_GM = getBAGSGMPercentage('Men','-','-')
WSA_BRA_KL_GM = getBAGSGMPercentage('Women','Bra','Lifestyle')
WSA_BRA_KR_GM = getBAGSGMPercentage('Women','Bra','Running')
WSA_BRA_KT_GM = getBAGSGMPercentage('Women','Bra','Training')
WSA_BRA_KY_GM = getBAGSGMPercentage('Women','Bra','Yoga')
WSA_BRA_GM    = getBAGSGMPercentage('Women','Bra','-')
WSA_JACKETS_KL_GM = getBAGSGMPercentage('Women','Jackets','Lifestyle')
WSA_JACKETS_KO_GM = getBAGSGMPercentage('Women','Jackets','Outdoor')
WSA_JACKETS_KR_GM = getBAGSGMPercentage('Women','Jackets','Running')
WSA_JACKETS_KT_GM = getBAGSGMPercentage('Women','Jackets','Training')
WSA_JACKETS_GM    = getBAGSGMPercentage('Women','Jackets','-')
WSA_SHORTS_KL_GM = getBAGSGMPercentage('Women','Shorts','Lifestyle')
WSA_SHORTS_KR_GM = getBAGSGMPercentage('Women','Shorts','Running')
WSA_SHORTS_KT_GM = getBAGSGMPercentage('Women','Shorts','Training')
WSA_SHORTS_KY_GM = getBAGSGMPercentage('Women','Shorts','Yoga')
WSA_SHORTS_GM    = getBAGSGMPercentage('Women','Shorts','-')
WSA_SWEATSHIRTS_KL_GM = getBAGSGMPercentage('Women','Sweatshirts','Lifestyle')
WSA_SWEATSHIRTS_KO_GM = getBAGSGMPercentage('Women','Sweatshirts','Outdoor')
WSA_SWEATSHIRTS_KR_GM = getBAGSGMPercentage('Women','Sweatshirts','Running')
WSA_SWEATSHIRTS_KT_GM = getBAGSGMPercentage('Women','Sweatshirts','Training')
WSA_SWEATSHIRTS_GM    = getBAGSGMPercentage('Women','Sweatshirts','-')
WSA_TIGHTS_KL_GM = getBAGSGMPercentage('Women','Tights','Lifestyle')
WSA_TIGHTS_KR_GM = getBAGSGMPercentage('Women','Tights','Running')
WSA_TIGHTS_KT_GM = getBAGSGMPercentage('Women','Tights','Training')
WSA_TIGHTS_KY_GM = getBAGSGMPercentage('Women','Tights','Yoga')
WSA_TIGHTS_GM    = getBAGSGMPercentage('Women','Tights','-')
WSA_TRACKPANTS_KL_GM = getBAGSGMPercentage('Women','Track Pants','Lifestyle')
WSA_TRACKPANTS_KO_GM = getBAGSGMPercentage('Women','Track Pants','Outdoor')
WSA_TRACKPANTS_KR_GM = getBAGSGMPercentage('Women','Track Pants','Running')
WSA_TRACKPANTS_KT_GM = getBAGSGMPercentage('Women','Track Pants','Training')
WSA_TRACKPANTS_KY_GM = getBAGSGMPercentage('Women','Track Pants','Yoga')
WSA_TRACKPANTS_GM    = getBAGSGMPercentage('Women','Track Pants','-')
WSA_TRACKSUITS_KL_GM = getBAGSGMPercentage('Women','Tracksuits','Lifestyle')
WSA_TRACKSUITS_KR_GM = getBAGSGMPercentage('Women','Tracksuits','Running')
WSA_TRACKSUITS_KT_GM = getBAGSGMPercentage('Women','Tracksuits','Training')
WSA_TRACKSUITS_KY_GM = getBAGSGMPercentage('Women','Tracksuits','Yoga')
WSA_TRACKSUITS_GM    = getBAGSGMPercentage('Women','Tracksuits','-')
WSA_TSHIRTS_KL_GM = getBAGSGMPercentage('Women','Tshirts','Lifestyle')
WSA_TSHIRTS_KO_GM = getBAGSGMPercentage('Women','Tshirts','Outdoor')
WSA_TSHIRTS_KR_GM = getBAGSGMPercentage('Women','Tshirts','Running')
WSA_TSHIRTS_KT_GM = getBAGSGMPercentage('Women','Tshirts','Training')
WSA_TSHIRTS_KY_GM = getBAGSGMPercentage('Women','Tshirts','Yoga')
WSA_TSHIRTS_GM    = getBAGSGMPercentage('Women','Tshirts','-')
WSA_GM = getBAGSGMPercentage('Women','-','-')
#########################################################
metrics = [[MSA_JACKETS_KL_CI,MSA_JACKETS_KL_OI,MSA_JACKETS_KL_CIV,MSA_JACKETS_KL_OIV,MSA_JACKETS_KL_STYLES_COUNT,MSA_JACKETS_KL_SALES_AMOUNT,MSA_JACKETS_KL_SALES_QUANTITY,MSA_JACKETS_KL_TD,MSA_JACKETS_KL_ASP,MSA_JACKETS_KL_GM],
[MSA_JACKETS_KO_CI,MSA_JACKETS_KO_OI,MSA_JACKETS_KO_CIV,MSA_JACKETS_KO_OIV,MSA_JACKETS_KO_STYLES_COUNT,MSA_JACKETS_KO_SALES_AMOUNT,MSA_JACKETS_KO_SALES_QUANTITY,MSA_JACKETS_KO_TD,MSA_JACKETS_KO_ASP,MSA_JACKETS_KO_GM],
[MSA_JACKETS_KR_CI,MSA_JACKETS_KR_OI,MSA_JACKETS_KR_CIV,MSA_JACKETS_KR_OIV,MSA_JACKETS_KR_STYLES_COUNT,MSA_JACKETS_KR_SALES_AMOUNT,MSA_JACKETS_KR_SALES_QUANTITY,MSA_JACKETS_KR_TD,MSA_JACKETS_KR_ASP,MSA_JACKETS_KR_GM],
[MSA_JACKETS_KT_CI,MSA_JACKETS_KT_OI,MSA_JACKETS_KT_CIV,MSA_JACKETS_KT_OIV,MSA_JACKETS_KT_STYLES_COUNT,MSA_JACKETS_KT_SALES_AMOUNT,MSA_JACKETS_KT_SALES_QUANTITY,MSA_JACKETS_KT_TD,MSA_JACKETS_KT_ASP,MSA_JACKETS_KT_GM],
[MSA_JACKETS_CI,MSA_JACKETS_OI   ,MSA_JACKETS_CIV   ,MSA_JACKETS_OIV   ,MSA_JACKETS_STYLES_COUNT   ,MSA_JACKETS_SALES_AMOUNT   ,MSA_JACKETS_SALES_QUANTITY   ,MSA_JACKETS_TD   ,MSA_JACKETS_ASP   ,MSA_JACKETS_GM   ],
[MSA_SHORTS_KL_CI,MSA_SHORTS_KL_OI,MSA_SHORTS_KL_CIV,MSA_SHORTS_KL_OIV,MSA_SHORTS_KL_STYLES_COUNT,MSA_SHORTS_KL_SALES_AMOUNT,MSA_SHORTS_KL_SALES_QUANTITY,MSA_SHORTS_KL_TD,MSA_SHORTS_KL_ASP,MSA_SHORTS_KL_GM],
[MSA_SHORTS_KR_CI,MSA_SHORTS_KR_OI,MSA_SHORTS_KR_CIV,MSA_SHORTS_KR_OIV,MSA_SHORTS_KR_STYLES_COUNT,MSA_SHORTS_KR_SALES_AMOUNT,MSA_SHORTS_KR_SALES_QUANTITY,MSA_SHORTS_KR_TD,MSA_SHORTS_KR_ASP,MSA_SHORTS_KR_GM],
[MSA_SHORTS_KT_CI,MSA_SHORTS_KT_OI,MSA_SHORTS_KT_CIV,MSA_SHORTS_KT_OIV,MSA_SHORTS_KT_STYLES_COUNT,MSA_SHORTS_KT_SALES_AMOUNT,MSA_SHORTS_KT_SALES_QUANTITY,MSA_SHORTS_KT_TD,MSA_SHORTS_KT_ASP,MSA_SHORTS_KT_GM],
[MSA_SHORTS_KY_CI,MSA_SHORTS_KY_OI,MSA_SHORTS_KY_CIV,MSA_SHORTS_KY_OIV,MSA_SHORTS_KY_STYLES_COUNT,MSA_SHORTS_KY_SALES_AMOUNT,MSA_SHORTS_KY_SALES_QUANTITY,MSA_SHORTS_KY_TD,MSA_SHORTS_KY_ASP,MSA_SHORTS_KY_GM],
[MSA_SHORTS_CI,MSA_SHORTS_OI   ,MSA_SHORTS_CIV   ,MSA_SHORTS_OIV   ,MSA_SHORTS_STYLES_COUNT   ,MSA_SHORTS_SALES_AMOUNT   ,MSA_SHORTS_SALES_QUANTITY   ,MSA_SHORTS_TD   ,MSA_SHORTS_ASP   ,MSA_SHORTS_GM   ],
[MSA_SWEATSHIRTS_KL_CI,MSA_SWEATSHIRTS_KL_OI,MSA_SWEATSHIRTS_KL_CIV,MSA_SWEATSHIRTS_KL_OIV,MSA_SWEATSHIRTS_KL_STYLES_COUNT,MSA_SWEATSHIRTS_KL_SALES_AMOUNT,MSA_SWEATSHIRTS_KL_SALES_QUANTITY,MSA_SWEATSHIRTS_KL_TD,MSA_SWEATSHIRTS_KL_ASP,MSA_SWEATSHIRTS_KL_GM],
[MSA_SWEATSHIRTS_KO_CI,MSA_SWEATSHIRTS_KO_OI,MSA_SWEATSHIRTS_KO_CIV,MSA_SWEATSHIRTS_KO_OIV,MSA_SWEATSHIRTS_KO_STYLES_COUNT,MSA_SWEATSHIRTS_KO_SALES_AMOUNT,MSA_SWEATSHIRTS_KO_SALES_QUANTITY,MSA_SWEATSHIRTS_KO_TD,MSA_SWEATSHIRTS_KO_ASP,MSA_SWEATSHIRTS_KO_GM],
[MSA_SWEATSHIRTS_KR_CI,MSA_SWEATSHIRTS_KR_OI,MSA_SWEATSHIRTS_KR_CIV,MSA_SWEATSHIRTS_KR_OIV,MSA_SWEATSHIRTS_KR_STYLES_COUNT,MSA_SWEATSHIRTS_KR_SALES_AMOUNT,MSA_SWEATSHIRTS_KR_SALES_QUANTITY,MSA_SWEATSHIRTS_KR_TD,MSA_SWEATSHIRTS_KR_ASP,MSA_SWEATSHIRTS_KR_GM],
[MSA_SWEATSHIRTS_KT_CI,MSA_SWEATSHIRTS_KT_OI,MSA_SWEATSHIRTS_KT_CIV,MSA_SWEATSHIRTS_KT_OIV,MSA_SWEATSHIRTS_KT_STYLES_COUNT,MSA_SWEATSHIRTS_KT_SALES_AMOUNT,MSA_SWEATSHIRTS_KT_SALES_QUANTITY,MSA_SWEATSHIRTS_KT_TD,MSA_SWEATSHIRTS_KT_ASP,MSA_SWEATSHIRTS_KT_GM],
[MSA_SWEATSHIRTS_CI,MSA_SWEATSHIRTS_OI   ,MSA_SWEATSHIRTS_CIV   ,MSA_SWEATSHIRTS_OIV   ,MSA_SWEATSHIRTS_STYLES_COUNT   ,MSA_SWEATSHIRTS_SALES_AMOUNT   ,MSA_SWEATSHIRTS_SALES_QUANTITY   ,MSA_SWEATSHIRTS_TD   ,MSA_SWEATSHIRTS_ASP   ,MSA_SWEATSHIRTS_GM   ],
[MSA_TRACKPANTS_KL_CI,MSA_TRACKPANTS_KL_OI,MSA_TRACKPANTS_KL_CIV,MSA_TRACKPANTS_KL_OIV,MSA_TRACKPANTS_KL_STYLES_COUNT,MSA_TRACKPANTS_KL_SALES_AMOUNT,MSA_TRACKPANTS_KL_SALES_QUANTITY,MSA_TRACKPANTS_KL_TD,MSA_TRACKPANTS_KL_ASP,MSA_TRACKPANTS_KL_GM],
[MSA_TRACKPANTS_KR_CI,MSA_TRACKPANTS_KR_OI,MSA_TRACKPANTS_KR_CIV,MSA_TRACKPANTS_KR_OIV,MSA_TRACKPANTS_KR_STYLES_COUNT,MSA_TRACKPANTS_KR_SALES_AMOUNT,MSA_TRACKPANTS_KR_SALES_QUANTITY,MSA_TRACKPANTS_KR_TD,MSA_TRACKPANTS_KR_ASP,MSA_TRACKPANTS_KR_GM],
[MSA_TRACKPANTS_KT_CI,MSA_TRACKPANTS_KT_OI,MSA_TRACKPANTS_KT_CIV,MSA_TRACKPANTS_KT_OIV,MSA_TRACKPANTS_KT_STYLES_COUNT,MSA_TRACKPANTS_KT_SALES_AMOUNT,MSA_TRACKPANTS_KT_SALES_QUANTITY,MSA_TRACKPANTS_KT_TD,MSA_TRACKPANTS_KT_ASP,MSA_TRACKPANTS_KT_GM],
[MSA_TRACKPANTS_KY_CI,MSA_TRACKPANTS_KY_OI,MSA_TRACKPANTS_KY_CIV,MSA_TRACKPANTS_KY_OIV,MSA_TRACKPANTS_KY_STYLES_COUNT,MSA_TRACKPANTS_KY_SALES_AMOUNT,MSA_TRACKPANTS_KY_SALES_QUANTITY,MSA_TRACKPANTS_KY_TD,MSA_TRACKPANTS_KY_ASP,MSA_TRACKPANTS_KY_GM],
[MSA_TRACKPANTS_CI,MSA_TRACKPANTS_OI   ,MSA_TRACKPANTS_CIV   ,MSA_TRACKPANTS_OIV   ,MSA_TRACKPANTS_STYLES_COUNT   ,MSA_TRACKPANTS_SALES_AMOUNT   ,MSA_TRACKPANTS_SALES_QUANTITY   ,MSA_TRACKPANTS_TD   ,MSA_TRACKPANTS_ASP   ,MSA_TRACKPANTS_GM   ],
[MSA_TRACKSUITS_KL_CI,MSA_TRACKSUITS_KL_OI,MSA_TRACKSUITS_KL_CIV,MSA_TRACKSUITS_KL_OIV,MSA_TRACKSUITS_KL_STYLES_COUNT,MSA_TRACKSUITS_KL_SALES_AMOUNT,MSA_TRACKSUITS_KL_SALES_QUANTITY,MSA_TRACKSUITS_KL_TD,MSA_TRACKSUITS_KL_ASP,MSA_TRACKSUITS_KL_GM],
[MSA_TRACKSUITS_KR_CI,MSA_TRACKSUITS_KR_OI,MSA_TRACKSUITS_KR_CIV,MSA_TRACKSUITS_KR_OIV,MSA_TRACKSUITS_KR_STYLES_COUNT,MSA_TRACKSUITS_KR_SALES_AMOUNT,MSA_TRACKSUITS_KR_SALES_QUANTITY,MSA_TRACKSUITS_KR_TD,MSA_TRACKSUITS_KR_ASP,MSA_TRACKSUITS_KR_GM],
[MSA_TRACKSUITS_KT_CI,MSA_TRACKSUITS_KT_OI,MSA_TRACKSUITS_KT_CIV,MSA_TRACKSUITS_KT_OIV,MSA_TRACKSUITS_KT_STYLES_COUNT,MSA_TRACKSUITS_KT_SALES_AMOUNT,MSA_TRACKSUITS_KT_SALES_QUANTITY,MSA_TRACKSUITS_KT_TD,MSA_TRACKSUITS_KT_ASP,MSA_TRACKSUITS_KT_GM],
[MSA_TRACKSUITS_KY_CI,MSA_TRACKSUITS_KY_OI,MSA_TRACKSUITS_KY_CIV,MSA_TRACKSUITS_KY_OIV,MSA_TRACKSUITS_KY_STYLES_COUNT,MSA_TRACKSUITS_KY_SALES_AMOUNT,MSA_TRACKSUITS_KY_SALES_QUANTITY,MSA_TRACKSUITS_KY_TD,MSA_TRACKSUITS_KY_ASP,MSA_TRACKSUITS_KY_GM],
[MSA_TRACKSUITS_CI,MSA_TRACKSUITS_OI   ,MSA_TRACKSUITS_CIV   ,MSA_TRACKSUITS_OIV   ,MSA_TRACKSUITS_STYLES_COUNT   ,MSA_TRACKSUITS_SALES_AMOUNT   ,MSA_TRACKSUITS_SALES_QUANTITY   ,MSA_TRACKSUITS_TD   ,MSA_TRACKSUITS_ASP   ,MSA_TRACKSUITS_GM   ],
[MSA_TSHIRTS_KL_CI,MSA_TSHIRTS_KL_OI,MSA_TSHIRTS_KL_CIV,MSA_TSHIRTS_KL_OIV,MSA_TSHIRTS_KL_STYLES_COUNT,MSA_TSHIRTS_KL_SALES_AMOUNT,MSA_TSHIRTS_KL_SALES_QUANTITY,MSA_TSHIRTS_KL_TD,MSA_TSHIRTS_KL_ASP,MSA_TSHIRTS_KL_GM],
[MSA_TSHIRTS_KO_CI,MSA_TSHIRTS_KO_OI,MSA_TSHIRTS_KO_CIV,MSA_TSHIRTS_KO_OIV,MSA_TSHIRTS_KO_STYLES_COUNT,MSA_TSHIRTS_KO_SALES_AMOUNT,MSA_TSHIRTS_KO_SALES_QUANTITY,MSA_TSHIRTS_KO_TD,MSA_TSHIRTS_KO_ASP,MSA_TSHIRTS_KO_GM],
[MSA_TSHIRTS_KR_CI,MSA_TSHIRTS_KR_OI,MSA_TSHIRTS_KR_CIV,MSA_TSHIRTS_KR_OIV,MSA_TSHIRTS_KR_STYLES_COUNT,MSA_TSHIRTS_KR_SALES_AMOUNT,MSA_TSHIRTS_KR_SALES_QUANTITY,MSA_TSHIRTS_KR_TD,MSA_TSHIRTS_KR_ASP,MSA_TSHIRTS_KR_GM],
[MSA_TSHIRTS_KT_CI,MSA_TSHIRTS_KT_OI,MSA_TSHIRTS_KT_CIV,MSA_TSHIRTS_KT_OIV,MSA_TSHIRTS_KT_STYLES_COUNT,MSA_TSHIRTS_KT_SALES_AMOUNT,MSA_TSHIRTS_KT_SALES_QUANTITY,MSA_TSHIRTS_KT_TD,MSA_TSHIRTS_KT_ASP,MSA_TSHIRTS_KT_GM],
[MSA_TSHIRTS_KY_CI,MSA_TSHIRTS_KY_OI,MSA_TSHIRTS_KY_CIV,MSA_TSHIRTS_KY_OIV,MSA_TSHIRTS_KY_STYLES_COUNT,MSA_TSHIRTS_KY_SALES_AMOUNT,MSA_TSHIRTS_KY_SALES_QUANTITY,MSA_TSHIRTS_KY_TD,MSA_TSHIRTS_KY_ASP,MSA_TSHIRTS_KY_GM],
[MSA_TSHIRTS_CI,MSA_TSHIRTS_OI   ,MSA_TSHIRTS_CIV   ,MSA_TSHIRTS_OIV   ,MSA_TSHIRTS_STYLES_COUNT   ,MSA_TSHIRTS_SALES_AMOUNT   ,MSA_TSHIRTS_SALES_QUANTITY   ,MSA_TSHIRTS_TD   ,MSA_TSHIRTS_ASP   ,MSA_TSHIRTS_GM   ],
[MSA_CI,MSA_OI,MSA_CIV,MSA_OIV,MSA_STYLES_COUNT,MSA_SALES_AMOUNT,MSA_SALES_QUANTITY,MSA_TD,MSA_ASP,MSA_GM],
[WSA_BRA_KL_CI,WSA_BRA_KL_OI,WSA_BRA_KL_CIV,WSA_BRA_KL_OIV,WSA_BRA_KL_STYLES_COUNT,WSA_BRA_KL_SALES_AMOUNT,WSA_BRA_KL_SALES_QUANTITY,WSA_BRA_KL_TD,WSA_BRA_KL_ASP,WSA_BRA_KL_GM],
[WSA_BRA_KR_CI,WSA_BRA_KR_OI,WSA_BRA_KR_CIV,WSA_BRA_KR_OIV,WSA_BRA_KR_STYLES_COUNT,WSA_BRA_KR_SALES_AMOUNT,WSA_BRA_KR_SALES_QUANTITY,WSA_BRA_KR_TD,WSA_BRA_KR_ASP,WSA_BRA_KR_GM],
[WSA_BRA_KT_CI,WSA_BRA_KT_OI,WSA_BRA_KT_CIV,WSA_BRA_KT_OIV,WSA_BRA_KT_STYLES_COUNT,WSA_BRA_KT_SALES_AMOUNT,WSA_BRA_KT_SALES_QUANTITY,WSA_BRA_KT_TD,WSA_BRA_KT_ASP,WSA_BRA_KT_GM],
[WSA_BRA_KY_CI,WSA_BRA_KY_OI,WSA_BRA_KY_CIV,WSA_BRA_KY_OIV,WSA_BRA_KY_STYLES_COUNT,WSA_BRA_KY_SALES_AMOUNT,WSA_BRA_KY_SALES_QUANTITY,WSA_BRA_KY_TD,WSA_BRA_KY_ASP,WSA_BRA_KY_GM],
[WSA_BRA_CI,WSA_BRA_OI   ,WSA_BRA_CIV   ,WSA_BRA_OIV   ,WSA_BRA_STYLES_COUNT   ,WSA_BRA_SALES_AMOUNT   ,WSA_BRA_SALES_QUANTITY   ,WSA_BRA_TD   ,WSA_BRA_ASP   ,WSA_BRA_GM   ],
[WSA_JACKETS_KL_CI,WSA_JACKETS_KL_OI,WSA_JACKETS_KL_CIV,WSA_JACKETS_KL_OIV,WSA_JACKETS_KL_STYLES_COUNT,WSA_JACKETS_KL_SALES_AMOUNT,WSA_JACKETS_KL_SALES_QUANTITY,WSA_JACKETS_KL_TD,WSA_JACKETS_KL_ASP,WSA_JACKETS_KL_GM],
[WSA_JACKETS_KO_CI,WSA_JACKETS_KO_OI,WSA_JACKETS_KO_CIV,WSA_JACKETS_KO_OIV,WSA_JACKETS_KO_STYLES_COUNT,WSA_JACKETS_KO_SALES_AMOUNT,WSA_JACKETS_KO_SALES_QUANTITY,WSA_JACKETS_KO_TD,WSA_JACKETS_KO_ASP,WSA_JACKETS_KO_GM],
[WSA_JACKETS_KR_CI,WSA_JACKETS_KR_OI,WSA_JACKETS_KR_CIV,WSA_JACKETS_KR_OIV,WSA_JACKETS_KR_STYLES_COUNT,WSA_JACKETS_KR_SALES_AMOUNT,WSA_JACKETS_KR_SALES_QUANTITY,WSA_JACKETS_KR_TD,WSA_JACKETS_KR_ASP,WSA_JACKETS_KR_GM],
[WSA_JACKETS_KT_CI,WSA_JACKETS_KT_OI,WSA_JACKETS_KT_CIV,WSA_JACKETS_KT_OIV,WSA_JACKETS_KT_STYLES_COUNT,WSA_JACKETS_KT_SALES_AMOUNT,WSA_JACKETS_KT_SALES_QUANTITY,WSA_JACKETS_KT_TD,WSA_JACKETS_KT_ASP,WSA_JACKETS_KT_GM],
[WSA_JACKETS_CI,WSA_JACKETS_OI   ,WSA_JACKETS_CIV   ,WSA_JACKETS_OIV   ,WSA_JACKETS_STYLES_COUNT   ,WSA_JACKETS_SALES_AMOUNT   ,WSA_JACKETS_SALES_QUANTITY   ,WSA_JACKETS_TD   ,WSA_JACKETS_ASP   ,WSA_JACKETS_GM   ],
[WSA_SHORTS_KL_CI,WSA_SHORTS_KL_OI,WSA_SHORTS_KL_CIV,WSA_SHORTS_KL_OIV,WSA_SHORTS_KL_STYLES_COUNT,WSA_SHORTS_KL_SALES_AMOUNT,WSA_SHORTS_KL_SALES_QUANTITY,WSA_SHORTS_KL_TD,WSA_SHORTS_KL_ASP,WSA_SHORTS_KL_GM],
[WSA_SHORTS_KR_CI,WSA_SHORTS_KR_OI,WSA_SHORTS_KR_CIV,WSA_SHORTS_KR_OIV,WSA_SHORTS_KR_STYLES_COUNT,WSA_SHORTS_KR_SALES_AMOUNT,WSA_SHORTS_KR_SALES_QUANTITY,WSA_SHORTS_KR_TD,WSA_SHORTS_KR_ASP,WSA_SHORTS_KR_GM],
[WSA_SHORTS_KT_CI,WSA_SHORTS_KT_OI,WSA_SHORTS_KT_CIV,WSA_SHORTS_KT_OIV,WSA_SHORTS_KT_STYLES_COUNT,WSA_SHORTS_KT_SALES_AMOUNT,WSA_SHORTS_KT_SALES_QUANTITY,WSA_SHORTS_KT_TD,WSA_SHORTS_KT_ASP,WSA_SHORTS_KT_GM],
[WSA_SHORTS_KY_CI,WSA_SHORTS_KY_OI,WSA_SHORTS_KY_CIV,WSA_SHORTS_KY_OIV,WSA_SHORTS_KY_STYLES_COUNT,WSA_SHORTS_KY_SALES_AMOUNT,WSA_SHORTS_KY_SALES_QUANTITY,WSA_SHORTS_KY_TD,WSA_SHORTS_KY_ASP,WSA_SHORTS_KY_GM],
[WSA_SHORTS_CI,WSA_SHORTS_OI   ,WSA_SHORTS_CIV   ,WSA_SHORTS_OIV   ,WSA_SHORTS_STYLES_COUNT   ,WSA_SHORTS_SALES_AMOUNT   ,WSA_SHORTS_SALES_QUANTITY   ,WSA_SHORTS_TD   ,WSA_SHORTS_ASP   ,WSA_SHORTS_GM   ],
[WSA_SWEATSHIRTS_KL_CI,WSA_SWEATSHIRTS_KL_OI,WSA_SWEATSHIRTS_KL_CIV,WSA_SWEATSHIRTS_KL_OIV,WSA_SWEATSHIRTS_KL_STYLES_COUNT,WSA_SWEATSHIRTS_KL_SALES_AMOUNT,WSA_SWEATSHIRTS_KL_SALES_QUANTITY,WSA_SWEATSHIRTS_KL_TD,WSA_SWEATSHIRTS_KL_ASP,WSA_SWEATSHIRTS_KL_GM],
[WSA_SWEATSHIRTS_KO_CI,WSA_SWEATSHIRTS_KO_OI,WSA_SWEATSHIRTS_KO_CIV,WSA_SWEATSHIRTS_KO_OIV,WSA_SWEATSHIRTS_KO_STYLES_COUNT,WSA_SWEATSHIRTS_KO_SALES_AMOUNT,WSA_SWEATSHIRTS_KO_SALES_QUANTITY,WSA_SWEATSHIRTS_KO_TD,WSA_SWEATSHIRTS_KO_ASP,WSA_SWEATSHIRTS_KO_GM],
[WSA_SWEATSHIRTS_KR_CI,WSA_SWEATSHIRTS_KR_OI,WSA_SWEATSHIRTS_KR_CIV,WSA_SWEATSHIRTS_KR_OIV,WSA_SWEATSHIRTS_KR_STYLES_COUNT,WSA_SWEATSHIRTS_KR_SALES_AMOUNT,WSA_SWEATSHIRTS_KR_SALES_QUANTITY,WSA_SWEATSHIRTS_KR_TD,WSA_SWEATSHIRTS_KR_ASP,WSA_SWEATSHIRTS_KR_GM],
[WSA_SWEATSHIRTS_KT_CI,WSA_SWEATSHIRTS_KT_OI,WSA_SWEATSHIRTS_KT_CIV,WSA_SWEATSHIRTS_KT_OIV,WSA_SWEATSHIRTS_KT_STYLES_COUNT,WSA_SWEATSHIRTS_KT_SALES_AMOUNT,WSA_SWEATSHIRTS_KT_SALES_QUANTITY,WSA_SWEATSHIRTS_KT_TD,WSA_SWEATSHIRTS_KT_ASP,WSA_SWEATSHIRTS_KT_GM],
[WSA_SWEATSHIRTS_CI,WSA_SWEATSHIRTS_OI   ,WSA_SWEATSHIRTS_CIV   ,WSA_SWEATSHIRTS_OIV   ,WSA_SWEATSHIRTS_STYLES_COUNT   ,WSA_SWEATSHIRTS_SALES_AMOUNT   ,WSA_SWEATSHIRTS_SALES_QUANTITY   ,WSA_SWEATSHIRTS_TD   ,WSA_SWEATSHIRTS_ASP   ,WSA_SWEATSHIRTS_GM   ],
[WSA_TIGHTS_KL_CI,WSA_TIGHTS_KL_OI,WSA_TIGHTS_KL_CIV,WSA_TIGHTS_KL_OIV,WSA_TIGHTS_KL_STYLES_COUNT,WSA_TIGHTS_KL_SALES_AMOUNT,WSA_TIGHTS_KL_SALES_QUANTITY,WSA_TIGHTS_KL_TD,WSA_TIGHTS_KL_ASP,WSA_TIGHTS_KL_GM],
[WSA_TIGHTS_KR_CI,WSA_TIGHTS_KR_OI,WSA_TIGHTS_KR_CIV,WSA_TIGHTS_KR_OIV,WSA_TIGHTS_KR_STYLES_COUNT,WSA_TIGHTS_KR_SALES_AMOUNT,WSA_TIGHTS_KR_SALES_QUANTITY,WSA_TIGHTS_KR_TD,WSA_TIGHTS_KR_ASP,WSA_TIGHTS_KR_GM],
[WSA_TIGHTS_KT_CI,WSA_TIGHTS_KT_OI,WSA_TIGHTS_KT_CIV,WSA_TIGHTS_KT_OIV,WSA_TIGHTS_KT_STYLES_COUNT,WSA_TIGHTS_KT_SALES_AMOUNT,WSA_TIGHTS_KT_SALES_QUANTITY,WSA_TIGHTS_KT_TD,WSA_TIGHTS_KT_ASP,WSA_TIGHTS_KT_GM],
[WSA_TIGHTS_KY_CI,WSA_TIGHTS_KY_OI,WSA_TIGHTS_KY_CIV,WSA_TIGHTS_KY_OIV,WSA_TIGHTS_KY_STYLES_COUNT,WSA_TIGHTS_KY_SALES_AMOUNT,WSA_TIGHTS_KY_SALES_QUANTITY,WSA_TIGHTS_KY_TD,WSA_TIGHTS_KY_ASP,WSA_TIGHTS_KY_GM],
[WSA_TIGHTS_CI,WSA_TIGHTS_OI   ,WSA_TIGHTS_CIV   ,WSA_TIGHTS_OIV   ,WSA_TIGHTS_STYLES_COUNT   ,WSA_TIGHTS_SALES_AMOUNT   ,WSA_TIGHTS_SALES_QUANTITY   ,WSA_TIGHTS_TD   ,WSA_TIGHTS_ASP   ,WSA_TIGHTS_GM   ],
[WSA_TRACKPANTS_KL_CI,WSA_TRACKPANTS_KL_OI,WSA_TRACKPANTS_KL_CIV,WSA_TRACKPANTS_KL_OIV,WSA_TRACKPANTS_KL_STYLES_COUNT,WSA_TRACKPANTS_KL_SALES_AMOUNT,WSA_TRACKPANTS_KL_SALES_QUANTITY,WSA_TRACKPANTS_KL_TD,WSA_TRACKPANTS_KL_ASP,WSA_TRACKPANTS_KL_GM],
[WSA_TRACKPANTS_KO_CI,WSA_TRACKPANTS_KO_OI,WSA_TRACKPANTS_KO_CIV,WSA_TRACKPANTS_KO_OIV,WSA_TRACKPANTS_KO_STYLES_COUNT,WSA_TRACKPANTS_KO_SALES_AMOUNT,WSA_TRACKPANTS_KO_SALES_QUANTITY,WSA_TRACKPANTS_KO_TD,WSA_TRACKPANTS_KO_ASP,WSA_TRACKPANTS_KO_GM],
[WSA_TRACKPANTS_KR_CI,WSA_TRACKPANTS_KR_OI,WSA_TRACKPANTS_KR_CIV,WSA_TRACKPANTS_KR_OIV,WSA_TRACKPANTS_KR_STYLES_COUNT,WSA_TRACKPANTS_KR_SALES_AMOUNT,WSA_TRACKPANTS_KR_SALES_QUANTITY,WSA_TRACKPANTS_KR_TD,WSA_TRACKPANTS_KR_ASP,WSA_TRACKPANTS_KR_GM],
[WSA_TRACKPANTS_KT_CI,WSA_TRACKPANTS_KT_OI,WSA_TRACKPANTS_KT_CIV,WSA_TRACKPANTS_KT_OIV,WSA_TRACKPANTS_KT_STYLES_COUNT,WSA_TRACKPANTS_KT_SALES_AMOUNT,WSA_TRACKPANTS_KT_SALES_QUANTITY,WSA_TRACKPANTS_KT_TD,WSA_TRACKPANTS_KT_ASP,WSA_TRACKPANTS_KT_GM],
[WSA_TRACKPANTS_KY_CI,WSA_TRACKPANTS_KY_OI,WSA_TRACKPANTS_KY_CIV,WSA_TRACKPANTS_KY_OIV,WSA_TRACKPANTS_KY_STYLES_COUNT,WSA_TRACKPANTS_KY_SALES_AMOUNT,WSA_TRACKPANTS_KY_SALES_QUANTITY,WSA_TRACKPANTS_KY_TD,WSA_TRACKPANTS_KY_ASP,WSA_TRACKPANTS_KY_GM],
[WSA_TRACKPANTS_CI,WSA_TRACKPANTS_OI   ,WSA_TRACKPANTS_CIV   ,WSA_TRACKPANTS_OIV   ,WSA_TRACKPANTS_STYLES_COUNT   ,WSA_TRACKPANTS_SALES_AMOUNT   ,WSA_TRACKPANTS_SALES_QUANTITY   ,WSA_TRACKPANTS_TD   ,WSA_TRACKPANTS_ASP   ,WSA_TRACKPANTS_GM   ],
[WSA_TRACKSUITS_KL_CI,WSA_TRACKSUITS_KL_OI,WSA_TRACKSUITS_KL_CIV,WSA_TRACKSUITS_KL_OIV,WSA_TRACKSUITS_KL_STYLES_COUNT,WSA_TRACKSUITS_KL_SALES_AMOUNT,WSA_TRACKSUITS_KL_SALES_QUANTITY,WSA_TRACKSUITS_KL_TD,WSA_TRACKSUITS_KL_ASP,WSA_TRACKSUITS_KL_GM],
[WSA_TRACKSUITS_KR_CI,WSA_TRACKSUITS_KR_OI,WSA_TRACKSUITS_KR_CIV,WSA_TRACKSUITS_KR_OIV,WSA_TRACKSUITS_KR_STYLES_COUNT,WSA_TRACKSUITS_KR_SALES_AMOUNT,WSA_TRACKSUITS_KR_SALES_QUANTITY,WSA_TRACKSUITS_KR_TD,WSA_TRACKSUITS_KR_ASP,WSA_TRACKSUITS_KR_GM],
[WSA_TRACKSUITS_KT_CI,WSA_TRACKSUITS_KT_OI,WSA_TRACKSUITS_KT_CIV,WSA_TRACKSUITS_KT_OIV,WSA_TRACKSUITS_KT_STYLES_COUNT,WSA_TRACKSUITS_KT_SALES_AMOUNT,WSA_TRACKSUITS_KT_SALES_QUANTITY,WSA_TRACKSUITS_KT_TD,WSA_TRACKSUITS_KT_ASP,WSA_TRACKSUITS_KT_GM],
[WSA_TRACKSUITS_KY_CI,WSA_TRACKSUITS_KY_OI,WSA_TRACKSUITS_KY_CIV,WSA_TRACKSUITS_KY_OIV,WSA_TRACKSUITS_KY_STYLES_COUNT,WSA_TRACKSUITS_KY_SALES_AMOUNT,WSA_TRACKSUITS_KY_SALES_QUANTITY,WSA_TRACKSUITS_KY_TD,WSA_TRACKSUITS_KY_ASP,WSA_TRACKSUITS_KY_GM],
[WSA_TRACKSUITS_CI,WSA_TRACKSUITS_OI   ,WSA_TRACKSUITS_CIV   ,WSA_TRACKSUITS_OIV   ,WSA_TRACKSUITS_STYLES_COUNT   ,WSA_TRACKSUITS_SALES_AMOUNT   ,WSA_TRACKSUITS_SALES_QUANTITY   ,WSA_TRACKSUITS_TD   ,WSA_TRACKSUITS_ASP   ,WSA_TRACKSUITS_GM   ],
[WSA_TSHIRTS_KL_CI,WSA_TSHIRTS_KL_OI,WSA_TSHIRTS_KL_CIV,WSA_TSHIRTS_KL_OIV,WSA_TSHIRTS_KL_STYLES_COUNT,WSA_TSHIRTS_KL_SALES_AMOUNT,WSA_TSHIRTS_KL_SALES_QUANTITY,WSA_TSHIRTS_KL_TD,WSA_TSHIRTS_KL_ASP,WSA_TSHIRTS_KL_GM],
[WSA_TSHIRTS_KO_CI,WSA_TSHIRTS_KO_OI,WSA_TSHIRTS_KO_CIV,WSA_TSHIRTS_KO_OIV,WSA_TSHIRTS_KO_STYLES_COUNT,WSA_TSHIRTS_KO_SALES_AMOUNT,WSA_TSHIRTS_KO_SALES_QUANTITY,WSA_TSHIRTS_KO_TD,WSA_TSHIRTS_KO_ASP,WSA_TSHIRTS_KO_GM],
[WSA_TSHIRTS_KR_CI,WSA_TSHIRTS_KR_OI,WSA_TSHIRTS_KR_CIV,WSA_TSHIRTS_KR_OIV,WSA_TSHIRTS_KR_STYLES_COUNT,WSA_TSHIRTS_KR_SALES_AMOUNT,WSA_TSHIRTS_KR_SALES_QUANTITY,WSA_TSHIRTS_KR_TD,WSA_TSHIRTS_KR_ASP,WSA_TSHIRTS_KR_GM],
[WSA_TSHIRTS_KT_CI,WSA_TSHIRTS_KT_OI,WSA_TSHIRTS_KT_CIV,WSA_TSHIRTS_KT_OIV,WSA_TSHIRTS_KT_STYLES_COUNT,WSA_TSHIRTS_KT_SALES_AMOUNT,WSA_TSHIRTS_KT_SALES_QUANTITY,WSA_TSHIRTS_KT_TD,WSA_TSHIRTS_KT_ASP,WSA_TSHIRTS_KT_GM],
[WSA_TSHIRTS_KY_CI,WSA_TSHIRTS_KY_OI,WSA_TSHIRTS_KY_CIV,WSA_TSHIRTS_KY_OIV,WSA_TSHIRTS_KY_STYLES_COUNT,WSA_TSHIRTS_KY_SALES_AMOUNT,WSA_TSHIRTS_KY_SALES_QUANTITY,WSA_TSHIRTS_KY_TD,WSA_TSHIRTS_KY_ASP,WSA_TSHIRTS_KY_GM],
[WSA_TSHIRTS_CI,WSA_TSHIRTS_OI   ,WSA_TSHIRTS_CIV   ,WSA_TSHIRTS_OIV   ,WSA_TSHIRTS_STYLES_COUNT   ,WSA_TSHIRTS_SALES_AMOUNT   ,WSA_TSHIRTS_SALES_QUANTITY   ,WSA_TSHIRTS_TD   ,WSA_TSHIRTS_ASP   ,WSA_TSHIRTS_GM   ],
[WSA_CI,WSA_OI,WSA_CIV,WSA_OIV,WSA_STYLES_COUNT,WSA_SALES_AMOUNT,WSA_SALES_QUANTITY,WSA_TD,WSA_ASP,WSA_GM],
]

df = pd.DataFrame(metrics,
             index=['MSA_JACKETS_KL','MSA_JACKETS_KO','MSA_JACKETS_KR','MSA_JACKETS_KT','MSA_JACKETS','MSA_SHORTS_KL','MSA_SHORTS_KR','MSA_SHORTS_KT',
                   'MSA_SHORTS_KY','MSA_SHORTS','MSA_SWEATSHIRTS_KL','MSA_SWEATSHIRTS_KO','MSA_SWEATSHIRTS_KR','MSA_SWEATSHIRTS_KT','MSA_SWEATSHIRTS',
                   'MSA_TRACKPANTS_KL','MSA_TRACKPANTS_KR','MSA_TRACKPANTS_KT','MSA_TRACKPANTS_KY','MSA_TRACKPANTS','MSA_TRACKSUITS_KL',
                   'MSA_TRACKSUITS_KR','MSA_TRACKSUITS_KT','MSA_TRACKSUITS_KY','MSA_TRACKSUITS','MSA_TSHIRTS_KL','MSA_TSHIRTS_KO','MSA_TSHIRTS_KR',
                   'MSA_TSHIRTS_KT','MSA_TSHIRTS_KY','MSA_TSHIRTS','MSA-TOTAL','WSA_BRA_KL','WSA_BRA_KR','WSA_BRA_KT','WSA_BRA_KY','WSA_BRA',
                   'WSA_JACKETS_KL','WSA_JACKETS_KO','WSA_JACKETS_KR','WSA_JACKETS_KT','WSA_JACKETS','WSA_SHORTS_KL','WSA_SHORTS_KR','WSA_SHORTS_KT',
                   'WSA_SHORTS_KY','WSA_SHORTS','WSA_SWEATSHIRTS_KL','WSA_SWEATSHIRTS_KO','WSA_SWEATSHIRTS_KR','WSA_SWEATSHIRTS_KT','WSA_SWEATSHIRTS',
                   'WSA_TIGHTS_KL','WSA_TIGHTS_KR','WSA_TIGHTS_KT','WSA_TIGHTS_KY','WSA_TIGHTS','WSA_TRACKPANTS_KL','WSA_TRACKPANTS_KO',
                   'WSA_TRACKPANTS_KR','WSA_TRACKPANTS_KT','WSA_TRACKPANTS_KY','WSA_TRACKPANTS','WSA_TRACKSUITS_KL','WSA_TRACKSUITS_KR',
                   'WSA_TRACKSUITS_KT','WSA_TRACKSUITS_KY','WSA_TRACKSUITS','WSA_TSHIRTS_KL','WSA_TSHIRTS_KO','WSA_TSHIRTS_KR','WSA_TSHIRTS_KT',
                   'WSA_TSHIRTS_KY','WSA_TSHIRTS','WSA-TOTAL'
],
             columns=['Current Inventory','Opening Inventory','Current Inventory Value','Opening Inventory Value','Styles','GMV','Sales Qty',
                      'TD%','ASP','GM%'])

df.to_excel('STD-MTD-AS-ON-31JAN2025.xlsx')